In [ ]:
%load_ext autoreload
%autoreload 2
#python v3.11.3

In [ ]:

import warnings
#to suppress seaborn palette warnings
warnings.filterwarnings("ignore", category=UserWarning)
#to suppress seaborn error estimation NaN warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)

import os
import datetime

import numpy as np #v1.25.0
import pandas as pd #v2.0.3
import scipy #v1.10.1
import scipy.stats as stats
from scipy.optimize import curve_fit

import seaborn as sns #v0.13.0
import matplotlib as mpl #v3.8.1
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import matplotlib.dates as mdates

#import matplotlib.spines as spines
from matplotlib.patches import Rectangle
from matplotlib.collections import LineCollection

import importlib
# these files should be in the same directory as this Jupyter notebook
from loader import *
import analysis_functions
from analysis_functions import *
from plot_functions import *

# to set styles for plotting
mpl.style.use(r"C:\Users\swind\Documents\lab_work\Barth\behavior_analysis\mouse_analysis\full_trial_analysis\mouse_analysis.mplstyle")

#to suppress inaccurate pandas reassignment warnings
pd.options.mode.chained_assignment = None  # default='warn'

In [ ]:
# directory files are pulled from
csv_directory = 'C:\\Users\\swind\\Documents\\lab_work\\Barth\\behavior_analysis\\Eunsol_data\\all_animals'
dcz_dir = 'C:\\Users\\swind\\Documents\\lab_work\\Barth\\behavior_analysis\\Eunsol_data\\all_animals\\dcz'
all_reward = 'C:\\Users\\swind\\Documents\\lab_work\\Barth\\behavior_analysis\\Eunsol_data\\all_animals\\all_reward'
rs_dir = r'C:\Users\swind\Documents\lab_work\Barth\behavior_analysis\Rachel_data\good'
new_acc = r'C:\Users\swind\Documents\lab_work\Barth\behavior_analysis\Eunsol_data\new vs old acc\new acc'
retrain_dir = r'C:\Users\swind\Documents\lab_work\Barth\behavior_analysis\retrain'
lesion_dir = r'C:\Users\swind\Documents\lab_work\Barth\behavior_analysis\lesion\Surgery done by Kate'
joe_dir = r'C:\Users\swind\Documents\lab_work\Barth\behavior_analysis\Joes C57 Training Data'

# file containing information about animals (cage, age, sex, strain) - must have Animal ID column
metadata_file = 'C:\\Users\\swind\\Documents\\lab_work\\Barth\\behavior_analysis\\Eunsol_data\\eunsol_metadata.xlsx'
dcz_meta = 'C:\\Users\\swind\\Documents\\lab_work\\Barth\\behavior_analysis\\Eunsol_data\\eunsol_metadata.xlsx'
rs_meta = r'C:\Users\swind\Documents\lab_work\Barth\behavior_analysis\Rachel_data\animal summary.xlsx'
new_meta = r'C:\Users\swind\Documents\lab_work\Barth\behavior_analysis\Eunsol_data\new vs old metadata.xlsx'
retrain_meta = r'C:\Users\swind\Documents\lab_work\Barth\behavior_analysis\retrain_metadata.xlsx'
lesion_meta = r'C:\Users\swind\Documents\lab_work\Barth\behavior_analysis\lesion\lesion metadata.xlsx'
joe_meta = r'C:\Users\swind\Documents\lab_work\Barth\behavior_analysis\joe_c57_metadata.xlsx'


mins = {
        "min_trials": 0,  # minimum number of trials in a bin to keep
        "min_blank": 0,   # minimum number of water trials
        "min_stimulus": 0 # minimum number of water trials 
        }


keys = {
    "millis bin":"puff delta",
    "Time (ms)":"Time (ms)",
    "stimulus type":"stimulus",
    "water delivery":"water",
    "trial no":"trial no",
    "time bin":"delta",
    "licks":"lick",
    "timestamp":"timestamp",
    "delay":"delay",
    "acc":"acc",
    'offset':'offset'
}

index = ["condition", "animal"]
keep = ["Sex", "acc", "Cage", 'Strain']
values = ["lick"]

sat = "#5E72FF"
dcz_hm4di = "#E1CD9B"
dcz_mcherry = "#B78787"
all_reward = "#B787B7"
ordr=["Control", "mCherry + CNO", "hM4Di + CNO","mCherry + DCZ", "hM4Di + DCZ", "100% ACC, 80% SAT"]
p=[sat, "maroon","goldenrod", dcz_mcherry, dcz_hm4di, all_reward]

In [ ]:
# metadata = pd.read_excel(metadata_file)
# dcz_metadata = pd.read_excel(dcz_meta)
# new_meta = pd.read_excel(new_meta)
# rs_meta = pd.read_excel(rs_meta)
lesion_metadata = pd.read_excel(lesion_meta)
retrain_metadata = pd.read_excel(retrain_meta)
joe_metadata = pd.read_excel(joe_meta)

# # this will load all conditions in a directory of multiple conditions
# ep = []
# for d in os.listdir(csv_directory):
#     print(f'starting {d}')
#     cond = make_condition_df(csv_directory +'\\' + d, d, metadata, keep)
#     ep.append(cond)

# print("starting DCZ")
# dcz = []
# for d in os.listdir(dcz_dir):
#     cond = make_condition_df(dcz_dir +'\\' + d, d, dcz_metadata, keep)
#     dcz.append(cond)

# print('starting new ACC')
# new = []
# for d in os.listdir(new_acc):
#     cond = make_condition_df(new_acc +'\\' + d, d, new_meta, keep)
#     new.append(cond)

# print('starting RS')
# rs = []
# for d in os.listdir(rs_dir):
#     cond = make_condition_df(rs_dir +'\\' + d, d, rs_meta, keep)
#     rs.append(cond)

# print('starting lesion')
# lesion = []
# for d in os.listdir(lesion_dir):
#     cond = make_condition_df(lesion_dir +'\\' + d, d, lesion_metadata, ['acc','id', 'Cage', 'Date', 'Sex'])
#     lesion.append(cond)

print('starting retrain')
retrain = []
for d in os.listdir(retrain_dir):
    if len(os.listdir(retrain_dir + '\\' + d)) > 0:
        an = make_condition_df(retrain_dir + '\\' + d, d, retrain_metadata, keep)
        retrain.append(an)
# joe = []
# print('starting joe c57')
# for d in os.listdir(joe_dir):
#     if 'SAT' in d:
#         cond = make_condition_df(joe_dir +'\\' + d, d, joe_metadata, keep)
#         joe.append(cond)


print("combine all")
df = pd.concat(retrain + lesion + joe) #pd.concat(ep + dcz + new + lesion + retrain)

keep = keep + ["stimulus", "water", "type"]
df = df.fillna('u')

In [ ]:
df["cond"] = df["condition"].replace({"hm4di_cno": "hM4Di + CNO", "mcherry_cno": "mCherry + CNO", 
                         '0.1mg DCZ hm4di': "hM4Di + DCZ", 
                         '0.1mg DCZ mcherry': "mCherry + DCZ", 
                         '0.5mg DCZ mcherry':"mCherry + DCZ",  
                         '0.5mg-1mg DCZ hm4di': "hM4Di + DCZ",
                         '100acc80acc':'100ACC80ACC',
                         "all_reward": "100ACC80SAT",
                         "SAT": "SAT", 'sat1':'SAT', 'sat5':'SAT', 
                         'acc2':'ACC', 'acc3':'ACC', 'acc7':'ACC',
                         'Ipsi-lesion':'Ipsi-lesion', 
                         'Contra-lesion':'Contra-lesion'})
keep = keep + ["cond"]

In [ ]:
# if there is a lick registered, make sure there is a poke registered
# in raw data, sometimes there are licks registered without pokes
df['pl'] = ((df['poke'] == 1) | (df['lick'] == 1)).replace({True:1, False:0})

In [ ]:
def pokestart(x):
    gppoke = x['pl'].diff().replace({-1:0})
    gppoke.iloc[0] = x.iloc[0]['pl']
    return gppoke
poke = df.groupby(['condition', 'animal', 'trial no']).apply(pokestart)
poke = poke.reset_index()['pl']
df['pokestart'] = poke
poke

### Inter-poke intervals

In [ ]:
# takes first sample of each trial and all samples where poke was initiated
pokestarts = pd.concat([df.groupby(['condition', 'animal', 'trial no']).first().reset_index(), df[df['pokestart'] == 1]])
# drops duplicates (most trial starts should have 1 for poke)
pokestarts = pokestarts.drop_duplicates()
pokestarts = pokestarts.sort_values(['condition', 'animal', 'trial no', 'timestamp'])
pkgp = pokestarts.groupby(['condition', 'animal'])
kpcols = list(pokestarts.columns.drop(['condition', 'animal']))
pokestarts['ipi'] = pkgp['timestamp'].diff()
pokestarts = pokestarts.dropna()
pkkeep = keep + ['ipi']
pokestarts['ipi'] = pokestarts["ipi"].to_numpy('timedelta64[ms]').astype(float)

# group and label with time (4 hour bin)
pokestarts = get_first_sample(pokestarts, index + ['trial no'], "timestamp", "trial start")
pokestarts = pokestarts.reset_index()
pokestarts = bin_by_time(pokestarts, 4*60, "min", index + ['trial no', 'index'], [], pkkeep + ['timestamp', 'offset'] + values, "trial start",
                       offset=pd.to_timedelta(12, unit="h"))
pokestarts = delta(pokestarts, index, "trial start", "acc", 'offset',"delta")
pokestarts = time_to_float(pokestarts, "Time (hr)", "delta", "m")
pokestarts["Time (hr)"] = pokestarts["Time (hr)"]/60.

# group and label with day
pokestarts = get_first_sample(pokestarts, index + ["trial no"], "timestamp", 'trial start')
pokestarts = bin_by_time(pokestarts, 1, "day", index + ['trial no', 'index'], [], 
                       pkkeep + values + ["timestamp", 'offset', "delta", "Time (hr)"], "trial start", 
                       offset=pd.to_timedelta(12, unit="h"))
pokestarts = delta(pokestarts, index, "trial start", 'acc', 'offset', "day delta")
pokestarts = time_to_float(pokestarts, "Day", "day delta", "D")
pokestarts = pokestarts[['condition','cond','animal', 'Day', 'Time (hr)', 'trial no', 'Sex', 'Strain', 'Cage', 'acc', 'stimulus', 'water', 'type', 'timestamp', 'lick', 'ipi']]

# get time from start of training from 4 hour bin
pokestarts = pokestarts.set_index(['condition', 'animal']) 
pokestarts['acc time'] = pokestarts.groupby(['condition', 'animal'])["Time (hr)"].transform(lambda x: x - x.iat[0])
pokestarts = pokestarts.reset_index()

In [ ]:
diffs_filt = pokestarts[(pokestarts['Time (hr)'] <= 48) & (pokestarts['Time (hr)'] >= -24)]
diffs_filt = diffs_filt.sort_values(['condition', 'animal', 'Time (hr)'])
tims = diffs_filt['Time (hr)'].unique()
tims.sort()
pal=sns.color_palette('Spectral', n_colors=len(tims))
conds = diffs_filt['condition'].unique()
for c in range(len(conds)):
    if 'control train' in conds[c]:
        filts = diffs_filt[diffs_filt['condition'] == conds[c]]
        ans = filts['animal'].unique()
        #ans.sort()
        f = plt.figure(figsize=(5*len(tims),5*len(ans)))
        subfigs = f.subfigures(len(ans), 1)
        print(conds[c], ans)
        for a in range(len(ans)):
            filtsan = diffs_filt[diffs_filt['animal'] == ans[a]]
            if len(ans) > 1:
                ax = subfigs[a].subplots(1, len(tims))
                subfigs[a].suptitle(f'{ans[a]}')
            else:
                ax = subfigs.subplots(1, len(tims))
                subfigs.suptitle(f'{ans[a]}')
            for n in range(len(tims)):
                filtstim = filtsan[filtsan['Time (hr)'] == tims[n]]
                if not (filtstim.empty):
                    ax[n].hist(filtstim['ipi'], color=pal[n],bins=range(2000, 7000, 200),zorder=n, alpha=0.8)
                    ax[n].set_xlabel('ipi (ms)')
                    ax[n].set_ylabel('Frequency')
                    ax[n].set_title(f'{tims[n]}', y=0.9)
            #f.legend(loc='center right', bbox_to_anchor=[1.5, 0.5])
    f.subplots_adjust(hspace=.3, wspace=.3)
    f.suptitle(conds[c],y=1.01)

In [ ]:
diffs_filt = pokestarts[pokestarts['Time (hr)'] <= 48]
diffs_filt = diffs_filt.sort_values(['condition', 'animal', 'Day'])
tims = diffs_filt['Day'].unique()
tims.sort()
pal=sns.color_palette('Spectral', n_colors=len(tims))
conds = diffs_filt['condition'].unique()
for c in range(len(conds)):
    filts = diffs_filt[diffs_filt['condition'] == conds[c]]
    if 'control train' in conds[c]:
        ans = filts['animal'].unique()
        #ans.sort()
        f = plt.figure(figsize=(5*len(tims),5*len(ans)))
        subfigs = f.subfigures(len(ans), 1)
        print(conds[c], ans)
        for a in range(len(ans)):
            filtsan = diffs_filt[diffs_filt['animal'] == ans[a]]
            if len(ans) > 1:
                ax = subfigs[a].subplots(1, len(tims))
                subfigs[a].suptitle(f'{ans[a]}')
            else:
                ax = subfigs.subplots(1, len(tims))
                subfigs.suptitle(f'{ans[a]}')
            for n in range(len(tims)):
                filtstim = filtsan[filtsan['Day'] == tims[n]]
                if not (filtstim.empty):
                    ax[n].hist(filtstim['ipi'], color=pal[n],bins=range(0, 10500, 200),zorder=n, alpha=0.8)
                    ax[n].set_xlabel('ipi (ms)')
                    ax[n].set_ylabel('Frequency')
                    ax[n].set_title(f'{tims[n]}', y=0.9)
            #f.legend(loc='center right', bbox_to_anchor=[1.5, 0.5])
        f.subplots_adjust(hspace=.3, wspace=.3)
        f.suptitle(conds[c],y=1.01)



#### Average by condition


Get bin counts by animal ACC


In [ ]:
histcounts = []
diffs_filt = pokestarts
acc = diffs_filt['acc time'].unique()
conds = diffs_filt['condition'].unique()
acc.sort()
for c in range(len(conds)):
    filts = diffs_filt[diffs_filt['condition'] == conds[c]]
    ans = filts['animal'].unique()
    condhistcounts = []
    condhistind = []
    condhistbin = []
    for a in range(len(ans)):
            filtsan = diffs_filt[diffs_filt['animal'] == ans[a]]
            if not filtsan.empty:
                for n in range(len(acc)):
                    filtstim = filtsan[filtsan['acc time'] == acc[n]]
                    if not (filtstim.empty):
                        counts, bins = np.histogram(filtstim['ipi'], bins=range(0, 10500, 200))
                        condhistind.append((conds[c], ans[a], acc[n]))
                        condhistcounts.append(counts)
                        condhistbin.append(bins[1:])
    condhistcounts = pd.DataFrame(condhistcounts, index=pd.MultiIndex.from_tuples(condhistind, names=['condition','animal', 'acc time'])).melt(var_name='bin no counts', value_name='ipi', ignore_index=False)
    condhistbin = pd.DataFrame(condhistbin, index=pd.MultiIndex.from_tuples(condhistind,names=['condition', 'animal', 'acc time'])).melt(var_name='bin no end', value_name='bin end', ignore_index=False)
    condhistcounts = pd.concat([condhistcounts, condhistbin], axis=1) 
    histcounts.append(condhistcounts)
histcountsACC = pd.concat(histcounts)
histcountsACC = histcountsACC.drop(columns='bin no end')

# get mean and SEM
gp = histcountsACC.groupby(['condition', 'acc time', 'bin no counts'])
histcountsACC = pd.concat([gp["ipi"].mean().rename("ipi mean"), gp["ipi"].sem().rename("ipi sem"), gp['bin end'].first() - 100], axis=1).reset_index()


Plot ACC


In [ ]:
ymax = {0:8, 4:15, 8:40, 12:40,16:40, 20:25, 24:20}
pal=sns.color_palette('Spectral', n_colors=10)
diffs_filt = histcountsACC[((histcountsACC['acc time'] <= 24) & (histcountsACC['acc time'] >= 0)) & (histcountsACC['condition'] == 'control train')]
diffs_filt = diffs_filt.sort_values('condition')
conds = diffs_filt['condition'].unique()
tims = diffs_filt['acc time'].unique()
tims.sort()
f, ax = plt.subplots(len(conds),len(tims), figsize=(5*len(tims), 5*len(conds)))
for c in range(len(conds)):
    print(conds[c])
    filts = diffs_filt[diffs_filt['condition'] == conds[c]]
    for n in range(len(tims)):
        filtstim = filts[filts['acc time'] == tims[n]]
        if not (filtstim.empty):
            l=''
            if n==0:
                    l=conds[c]
            ax[n].bar(x=filtstim['bin end'], height=filtstim['ipi mean'],
                        yerr=filtstim['ipi sem'],width=200, color=pal[n],alpha=1,
                        error_kw={'lw':0.2, 'capsize':0.8,'lolims':True}, zorder=7-n, label=l)
            ax[n].set_title(f'{conds[c]} {tims[n]}', y=0.9)
            
#f.legend()
xmaj = ticker.MultipleLocator(2500) 
xmin = ticker.MultipleLocator(500) 
ymaj = ticker.MultipleLocator(50) 
ymin = ticker.MultipleLocator(25) 
for ax in f.axes:
    style_axes_helper(ax, ax.get_title(),'ipi (ms)','Frequency', [0, 10500], [0, 250], xmaj, xmin, ymaj, ymin)
f.subplots_adjust(hspace=.35, wspace=.35)
f.suptitle('First 24h',y=0.95)

In [ ]:
ymax = {0:8, 4:15, 8:40, 12:40,16:40, 20:25, 24:20}
pal=['black', 'darkblue', 'pink']
diffs_filt = histcountsACC[((histcountsACC['condition'].str.contains('lesion'))|(histcountsACC['condition']=='retrain - train')) & ((histcountsACC['acc time'] <= 72) & (histcountsACC['acc time'] >= 0))]
diffs_filt = diffs_filt.sort_values('condition')
conds = diffs_filt['condition'].unique()
tims = diffs_filt['acc time'].unique()
tims.sort()
f, ax = plt.subplots(1,len(tims), figsize=(5*len(tims), 5))
for c in range(len(conds)):
    print(conds[c])
    filts = diffs_filt[diffs_filt['condition'] == conds[c]]
    for n in range(len(tims)):
        filtstim = filts[filts['acc time'] == tims[n]]
        if not (filtstim.empty):
            l=''
            if n==0:
                l=conds[c]
            ax[n].bar(x=filtstim['bin end'], height=filtstim['ipi mean'],
                        yerr=filtstim['ipi sem'],width=200, color=pal[c],alpha=0.5,
                        error_kw={'lw':0.2, 'capsize':0.8,'lolims':True}, zorder=7-n, label=l)
            ax[n].set_title(f'{conds[c]} {tims[n]}', y=0.9)
            
f.legend()
xmaj = ticker.MultipleLocator(2500) 
xmin = ticker.MultipleLocator(500) 
ymaj = ticker.MultipleLocator(50) 
ymin = ticker.MultipleLocator(25) 
for ax in f.axes:
    style_axes_helper(ax, ax.get_title(),'ipi (ms)','Frequency', [0, 10500], [0, 250], xmaj, xmin, ymaj, ymin)
f.subplots_adjust(hspace=.35, wspace=.35)
f.suptitle('First 24h',y=0.95)

In [ ]:
conds = histcountsACC["condition"].unique()
tcond = histcountsACC["condition"] == 'retrain - train'
rcond = histcountsACC["condition"] == 'retrain - retrain'
filt = (histcountsACC['condition'].str.contains('retrain'))
ind = histcountsACC[filt][["acc time"]].apply(tuple, axis=1).unique()

res=[]
for t in ind:
    daycond = histcountsACC["acc time"] == t
    tr = histcountsACC[filt & daycond & tcond]["ipi mean"]
    rtr = histcountsACC[filt & daycond & rcond]["ipi mean"]
    res.append(stats.ks_2samp(tr, rtr))

pd.DataFrame(res, columns=["statistic", "pvalue"], index=pd.Series(ind).apply(lambda x: 'acc time : ' + str(x[0])))

Get bin counts by animal SAT


In [ ]:
pal=sns.color_palette('Spectral', n_colors=10)
diffs_filt = pokestarts[((pokestarts['Time (hr)'] >= 0))]
conds = diffs_filt['condition'].unique()
tims = diffs_filt['Time (hr)'].unique()
st = diffs_filt['type'].unique()
tims.sort()
histcounts = []
for c in range(len(conds)):
    filts = diffs_filt[diffs_filt['condition'] == conds[c]]
    ans = filts['animal'].unique()
    print(ans)
    condhistcounts = []
    condhistind = []
    condhistbin = []
    for a in range(len(ans)):
        filtsan = diffs_filt[diffs_filt['animal'] == ans[a]]
        for n in range(len(tims)):
            filtstim = filtsan[filtsan['Time (hr)'] == tims[n]]
            for s in range(len(st)):
                filtsst = filtstim[filtstim['type'] == st[s]]
                if not (filtsst.empty):
                    counts, bins = np.histogram(filtsst['ipi'], bins=range(0, 10500, 500))
                    condhistind.append((conds[c],ans[a], tims[n], st[s]))
                    condhistcounts.append(counts)
                    condhistbin.append(bins[1:])
    condhistcounts = pd.DataFrame(condhistcounts, index=pd.MultiIndex.from_tuples(condhistind, names=['condition','animal', 'Time (hr)', 'type'])).melt(var_name='bin no counts', value_name='ipi', ignore_index=False)
    condhistbin = pd.DataFrame(condhistbin, index=pd.MultiIndex.from_tuples(condhistind,names=['condition','animal', 'Time (hr)', 'type'])).melt(var_name='bin no end', value_name='bin end', ignore_index=False)
    condhistcounts = pd.concat([condhistcounts, condhistbin], axis=1) 
    histcounts.append(condhistcounts)
histcountsSAT = pd.concat(histcounts)
histcountsSAT = histcountsSAT.drop(columns='bin no end')

gp = histcountsSAT.groupby(['condition','Time (hr)', 'type','bin no counts'])
histcountsSAT = pd.concat([gp["ipi"].mean().rename("ipi mean"), gp["ipi"].sem().rename("ipi sem"), gp['bin end'].first() - 250], axis=1).reset_index()


Plot SAT


In [ ]:
#pal=sns.color_palette('Spectral', n_colors=7)
pal = ["red", 'green']#,"mistyrose", "lightgreen"]

diffs_filt = histcountsSAT[(histcountsSAT['condition'].str.contains('control')) & ((histcountsSAT['Time (hr)'] <= 24) & (histcountsSAT['Time (hr)'] >= 0))]
conds = diffs_filt['condition'].unique()
tims = diffs_filt['Time (hr)'].unique()
st = diffs_filt['type'].unique()
f, ax = plt.subplots(len(conds),len(tims), figsize=(5*len(tims), 5*len(conds)))
for c in range(len(conds)):
    print(conds[c])
    filts = diffs_filt[diffs_filt['condition'] == conds[c]]
    for n in range(len(tims)):
        filtstim = filts[filts['Time (hr)'] == tims[n]]
        for s in range(len(st)):
            filtst = filtstim[filtstim['type'] == st[s]]
            if not (filtst.empty):
                ax[c][n].bar(x=filtst['bin end'], height=filtst['ipi mean'],
                            yerr=filtst['ipi sem'],width=500, color=pal[s],alpha=1,
                            error_kw={'lw':0.2, 'capsize':0.8,'lolims':True}, zorder=2-s)
                ax[c][n].set_xlabel('ipi (ms)')
                ax[c][n].set_ylabel('Frequency')
                ax[c][n].set_title(f'{conds[c]}', y=0.9)
                ax[c][n].set_ylim([0, 300])
                ax[c][n].set_xlim([0, 10000])
                    
#f.legend()
f.subplots_adjust(hspace=.3, wspace=.3)
f.suptitle('SAT24',y=1.01)


In [ ]:
pal=['black', 'darkblue', 'pink']
diffs_filt = histcountsSAT[((histcountsSAT['condition'].str.contains('lesion')) |(histcountsSAT['condition'] == 'retrain - train'))&((histcountsSAT['Time (hr)'] <= 48) & (histcountsSAT['Time (hr)'] >= 0))]
ymax = {0:8, 4:15, 8:40, 12:40,16:40, 20:25, 24:20}
conds = diffs_filt['condition'].unique()
tims = diffs_filt['Time (hr)'].unique()
f, ax = plt.subplots(1,len(tims), figsize=(5*len(tims), 5))
for c in range(len(conds)):
    print(conds[c])
    filts = diffs_filt[diffs_filt['condition'] == conds[c]]
    for n in range(len(tims)):
        filtstim = filts[filts['Time (hr)'] == tims[n]]
        if not (filtstim.empty):
            if n==1:
                    l=conds[c]
            else: 
                l=''
            ax[n].bar(x=filtstim['bin end'], height=filtstim['ipi mean'],
                        yerr=filtstim['ipi sem'],width=200, color=pal[c],alpha=0.5,
                        error_kw={'lw':0.2, 'capsize':0.8,'lolims':True}, zorder=7-n, label=l)
            ax[n].set_xlabel('ipi (ms)')
            ax[n].set_ylabel('Frequency')
            ax[n].set_title(f'{tims[n]}', y=0.9)
            ax[n].set_ylim([0, 200])
        
f.legend()
f.subplots_adjust(hspace=.3, wspace=.3)
f.suptitle('SAT 24',y=1.01)


In [ ]:
conds = histcountsSAT["condition"].unique()
tcond = histcountsSAT["condition"] == 'retrain - train'
rcond = histcountsSAT["condition"] == 'retrain - retrain'
filt = (histcountsSAT['condition'].str.contains('retrain'))
ind = histcountsSAT[filt][["Time (hr)"]].apply(tuple, axis=1).unique()

res=[]
for t in ind:
    daycond = histcountsSAT["Time (hr)"] == t
    tr = histcountsSAT[filt & daycond & tcond]["ipi mean"]
    rtr = histcountsSAT[filt & daycond & rcond]["ipi mean"]
    res.append(stats.ks_2samp(tr, rtr))

pd.DataFrame(res, columns=["statistic", "pvalue"], index=pd.Series(ind).apply(lambda x: 'acc time : ' + str(x[0])))

Pokecount over time and pokes/trial

In [ ]:
kp = ['Day', 'Sex', 'Cage', 'ipi', 'lick', 'acc time', 'Strain']
counts = trial_counts(pokestarts, ['condition', 'animal', 'Time (hr)'], kp,'trial no')
# pokestarts is only samples where poke is initiated
# so number of samples per trial is equivalent to number of poke initiations
pokesfrac = pokestarts.groupby(['condition', 'animal', 'Time (hr)', 'trial no'])['timestamp'].nunique()
pkkp = pokestarts.groupby(['condition', 'animal', 'Time (hr)', 'trial no']).first()[kp]
pokesfrac = pd.concat([pokesfrac, pkkp], axis=1).reset_index()

In [ ]:
g = plot_hr(pokesfrac[(pokesfrac['condition'].str.contains('control train'))], ymajmult=2, yminmult=1,
            y='timestamp', hue='animal',
            ylim=[0, 5],xlim=[-24, 48],
            errorbar='se', err_style='bars', err_kws={'lw':1,'capsize':0.8}, palette='grey',legend='full')
for (name, ax) in g.axes_dict.items():
    if name == 'SAT' or name == 'SATnew' or name == 'all_reward' or 'cno' in name:
        ax.set_xlim([-24, 24])
    if name=='acc3':
        ax.set_ylim([0, 600])
g.fig.suptitle('pokes per trial', y=1.01, fontsize=20)
g.axes[0][0].axhline(y=3, xmin=0, xmax=1, ls="-", color="black", zorder=10, lw=1)


In [ ]:
g = plot_hr(counts[(counts['condition'].str.contains('control train'))], ymajmult=400, yminmult=100,
            y='trial no', hue='condition',
            ylim=[0, 800],xlim=[-24, 48],
            errorbar='se', err_style='bars', err_kws={'lw':1,'capsize':0.8}, palette='husl',legend='full')
for (name, ax) in g.axes_dict.items():
    if name == 'SAT' or name == 'SATnew' or name == 'all_reward' or 'cno' in name:
        ax.set_xlim([-24, 24])
    if name=='acc3':
        ax.set_ylim([0, 600])
g.fig.suptitle('all trials', y=1.01, fontsize=20)


In [ ]:
ipi_measures = pokestarts.set_index(['condition', 'animal', 'trial no'])
#ITI_measures = ITI_measures[(ITI_measures['ITI'] <= 10000)]
ipi_measures = ipi_measures.groupby(['condition','animal', 'Day', 'Time (hr)'])['ipi'].apply(lambda x: pd.Series([stats.skew(x), stats.kurtosis(x), x.mean(),x.median(),x.mode().iat[0],x.std(), x.sem()], index=['skew', 'kurtosis', 'mean','median','mode','stdev', 'sem'], name='measures'))
ipi_measures = ipi_measures.reset_index().dropna()
ipi_measures = ipi_measures.rename(columns={'level_4':'measures'})
ipi_measures['acc time'] = ipi_measures.groupby(['condition', 'animal'])["Time (hr)"].transform(lambda x: x - x.iat[0])
ipi_measures["Time (hr)"] = ipi_measures["Time (hr)"] + 2

In [ ]:
colors = sns.color_palette('husl', n_colors=2)
#for c in ITI_measures['condition'].unique():
g = sns.relplot(ipi_measures[(ipi_measures['condition'].str.contains('control train')) & (ipi_measures["Time (hr)"] <=120)],
            x='Time (hr)', y='ipi', kind='line',
            col='measures', col_wrap=3,hue='condition',col_order=['skew', 'kurtosis', 'mean','stdev', 'median', 'mode'], errorbar='se', err_style='bars', err_kws={'lw':0.2}, 
            marker='o', ms='8', mec=None, lw=1, palette=['black','lightblue','mediumblue', 'black'],facet_kws={'sharey':False})
for(name, ax) in g.axes_dict.items():
        ax.set_title(name)
        ax.set_xlim([-24,48])
        ax.xaxis.set_major_locator(ticker.MultipleLocator(24))
        ax.xaxis.set_minor_locator(ticker.MultipleLocator(6))
        #ax.set_xticklabels(ax.get_xticklabels(), rotation=90, fontsize=18)
        
        # if name in ['mean']:
        #     ax.set_ylim([0,1500000])
        # if name in ['median']:
        #     ax.set_ylim([0,12000])
        # if name in ['mode']:
        #     ax.set_ylim([0,1000])
        # if name == 'stdev':
        #     ax.set_ylim([0, 2000000])
g.fig.suptitle('ipi measures', y=1.05)


In [ ]:
(df[df['condition code'] == 5].groupby(['condition', 'animal', 'trial no']).first()['timestamp'] - df[df['condition code'] == 4].groupby(['condition', 'animal', 'trial no']).last()['timestamp']).groupby(['condition', 'animal']).mean()

### Poke frequency across trial

In [ ]:
freq_window = None # milliseconds, length of rolling window
freq_bin = 100 # milliseconds, length of discrete time bin
keys["millis bin"] = "puff delta"
time_bin = 4*60 # minutes
keys['licks'] = 'poke'
data = full_lickfreq(df[df['condition'] == 'control train'], freq_window, freq_bin, time_bin, keys, keep, ['lick','poke'])
data['poke'] = data['poke']*0.1
data['trial start'] = data.groupby(['condition', 'animal', 'trial no'])["Time (ms)"].transform(lambda x: x - x.iat[0])

flkeep = keep + ["Time (hr)", "delta", "Day", "day_delta", "puff delta", "Time (ms)", 'trial start']

keys["time bin"] = "delta"
means, flcounts, perf = agg(data, keys, index, flkeep, ['lick','poke'])
daytots = sum_trials(flcounts, index + ["Day"],[], "trial no")
hourtots = sum_trials(flcounts, index + ["Time (hr)"],[], "trial no")

mins["min_trials"] = 10 # minimum number of trials in a bin to keep
mins["min_blank"] = 1 # minimum number of water trials
mins["min_stimulus"] = 1 # minimum number of water trials

data_filtered = drop_group(data, mins, keys, "trial no", index, flkeep)
filtmeans, flfiltcounts, filtperf = agg(data_filtered, keys, index, flkeep, ['lick','poke'])
daytotsfilt = sum_trials(flfiltcounts, index + ["Day"],[], "trial no")
hourtotsfilt = sum_trials(flfiltcounts, index + ["Time (hr)"], [],"trial no")

In [ ]:
for c in filtmeans['condition'].unique():
    if 'control' in c:
        for a in filtmeans[(filtmeans['condition'] == c)]['animal'].unique():
                plot_lickfreq(filtmeans[(filtmeans['animal'] == a)], y='pl',suptitle=f'{c}: {a}',
                            col_wrap=6,errorbar='se', err_style='bars', err_kws={'lw':0.7}, xlim=[-800, 2000])

In [ ]:
for c in filtmeans['condition'].unique():
    if 'control' in c:
        g = plot_lickfreq(filtmeans[(filtmeans['condition'] == c)& (filtmeans['Time (hr)'] >= -24)], x='Time (ms)',y='lick',
                                suptitle='lick control train', col_wrap=6,errorbar='se',
                                #xlabel='Time from trial initiation (ms)',
                                err_style='bars', err_kws={'lw':0.7}, ylim=[0,12],xlim=[1500, 3000],
                                style=True)
        for (name,ax) in g.axes_dict.items(): 
            ax.axvline(x=2000, ymin=0, ymax=1, ls="-", color="black", zorder=10, lw=1)

In [ ]:
for c in filtmeans['condition'].unique():
    if 'control' in c:
        g = plot_lickfreq(filtmeans[(filtmeans['condition'] == c)& (filtmeans['Time (hr)'] >= -24)], x='Time (ms)',y='poke',
                                suptitle='poke control train', col_wrap=6,errorbar='se', ylabel='Poke Frequency',
                                #xlabel='Time from trial initiation (ms)',
                                err_style='bars', err_kws={'lw':0.7}, ylim=[0,1.1],xlim=[1500, 3000],
                                ymajmult=0.1, style=True)
        for (name,ax) in g.axes_dict.items(): 
            ax.axvline(x=2000, ymin=0, ymax=1, ls="-", color="black", zorder=10, lw=1)

### Inter-trial intervals

In [ ]:
# get first sample of each trial
firsts = df.groupby(["condition", "animal", "trial no"]).first()
last = df.groupby(["condition", "animal", "trial no"]).last()
puffstart = df[df['delay'] == pd.to_timedelta(0)].groupby(["condition", "animal", "trial no"]).first()

In [ ]:
pf.head()

In [ ]:
lf.head()

In [ ]:
#puff onset to end of trial
pl = last['timestamp'] - puffstart['timestamp']
pl = pl.rename('puff to end')

#puff onset to start of next trial
pf = puffstart['timestamp'] - firsts.groupby(['condition','animal']).shift()['timestamp']
pf = pf.rename('puff to next')
#trial end to next trial onset
lf = last['timestamp'] - firsts.groupby(['condition','animal']).shift()['timestamp']
lf = lf.rename('end to next')
diffs = pd.concat([pl, pf, lf], axis=1).reset_index()
diffs

In [ ]:
# get first sample of each trial
firsts = df.groupby(["condition", "animal", "trial no"]).first().reset_index()
last = df.groupby(["condition", "animal", "trial no"]).last().reset_index()

# get 1st through 10th ITI (current trial to previous trial) for each trial
# continuous ITI across all trials
diffs = []
for i in range(1, 10):
    kpcols = list(firsts.columns.drop(['condition', 'animal', 'trial no']))
    gp =  firsts.sort_values(['condition', 'animal', 'trial no']).set_index(['condition', 'animal']).groupby(['condition', 'animal'])
    dif = gp['timestamp'].diff(i).rename(i)
    diffs.append(dif)
kp = firsts.groupby(['condition', 'animal', 'trial no'])[kpcols].first().reset_index().set_index(['condition', 'animal'])
diffs_by_time = pd.concat([kp] + diffs, axis=1)
# all ITI columns together
diffs_by_time = diffs_by_time.melt(id_vars=kpcols + ['trial no'],value_vars = range(1,10), var_name='diff num', value_name='ITI',ignore_index=False)
# drop NaN ITI (first n columns per animal for nth ITI - no preceding trials)
diffs_by_time = diffs_by_time.dropna().reset_index()
# convert to float
diffs_by_time['ITI'] = diffs_by_time["ITI"].to_numpy('timedelta64[ms]').astype(float)

In [ ]:
keep = keep + ['diff num', 'ITI']
# group and label with time (4 hour bin)
diffs_by_time = get_first_sample(diffs_by_time, index + ['trial no'], "timestamp", "trial start")
diffs_by_time = bin_by_time(diffs_by_time, 4*60, "min", index + ['trial no'], [], keep + ['timestamp', 'offset'] + values, "trial start",
                       offset=pd.to_timedelta(12, unit="h"))
diffs_by_time = delta(diffs_by_time, index, "trial start", "acc", 'offset',"delta")
diffs_by_time = time_to_float(diffs_by_time, "Time (hr)", "delta", "m")
diffs_by_time["Time (hr)"] = diffs_by_time["Time (hr)"]/60.

# group and label with day
diffs_by_time = get_first_sample(diffs_by_time, index + ["trial no"], "timestamp", 'trial start')
diffs_by_time = bin_by_time(diffs_by_time, 1, "day", index + ['trial no'], [], 
                       keep + values + ["timestamp", 'offset', "delta", "Time (hr)"], "trial start", 
                       offset=pd.to_timedelta(12, unit="h"))
diffs_by_time = delta(diffs_by_time, index, "trial start", 'acc', 'offset', "day delta")
diffs_by_time = time_to_float(diffs_by_time, "Day", "day delta", "D")
diffs_by_time = diffs_by_time[['condition', 'animal', 'Day', 'Time (hr)', 'trial no', 'Sex', 'Strain', 'Cage', 'acc', 'stimulus', 'water', 'timestamp', 'lick', 'diff num', 'ITI']]

# get time from start of training from 4 hour bin
diffs_by_time = diffs_by_time.set_index(['condition', 'animal']) 
diffs_by_time['acc time'] = diffs_by_time.groupby(['condition', 'animal'])["Time (hr)"].transform(lambda x: x - x.iat[0])
diffs_by_time = diffs_by_time.reset_index()

In [ ]:
### Get learn type
timing = {4:  ['NPG6', 'NKZ7', 'NLG10','PIT9', 'NLG3'], 
          8:  ['MYS7','MBR1','NGZ1', 'NKZ10', 'NLB10', 'NLN5','PLX11','OGV1', "OKK4", 'GXS4'], 
          12: ['LYO1','LYO3','MZI1', 'NPG3','PLX14','PLC2', 'GSB8','OEP2','PIT10'], 
          16: ['MBR3','NPG1','NRM3','LYO2', 'NKZ4', 'NLB1','PLC7','PLC9','PLC4','PLW5', 'JSS4', 'NFZ13','OEP5', 'OIJ6', "MKV10", 'PIE4', 'PGL6'],
          20: ['NPG5', 'MMY10','NLB3','NLV3','PLC6','NJO1', 'LWP8', "NBX7", 'PIT17'],
          24: [],
          28: ['NRM5', 'NPG2', 'NLV2','OEX5', 'OGV2','OEX8','PLW6','PLX13', 'ONW8-recheck training', 'LWP2', 'LWP4', 'OHQ5','OHQ6','OKK7', 'NVN2', 'PIT15'],
          32: ['NGZ5','NEL3', 'NKZ3', 'NLN3','PLW2', 'OPY5', 'MKV11', 'MJN3', 'NBE11','OWY6', 'NHR1'],
          36: ['MMY9','NEL1', 'MMY7','JND2', 'LWP6', 'MJL2', 'MJN2', 'NCY1', "NIO15", 'OHQ3', 'PIT11'],
          40: ['NLB8','NLV5','OEX6', 'NGX5'],
          44: ['MKV14'],
          48: [],
          'wrong': ['NFZ4', 'NAS9', 'NBX9', 'NJW5', 'MJN5', 'MYC4', 'MZG4','MZG5', 'NCY4',
                   'NCY6', 'NDR1', 'NIO16', "OSA10", 'OSA8', 'OWY5', 'OGY2', 'OKN2',
                   'PGL5', 'PIE8', 'PIT14', 'NEJ1'],
          'nd': ['MYS1', 'NEL5', 'NHR7', 'MYS2', "NRM2", 'NEL2', 'GXS5','OSA12' ]
          }
types = {"fast": ['LYO1', 'MYS7', 'NPG6', 'LYO3', 'MBR1', 'MZI1', 'NGZ1', 
                  'NPG3', 'NKZ10', 'NKZ7', 'NLB10', 'NLG3', 'NLN5', 'PLX11',
                  'PLX14','PLC2', 'OGV1', 'GSB8', "GXS4",'OEP2', 'OKK4', 'PIT10', 'PIT9'],
         "med": ['MBR3', 'NPG1', 'NRM3', 'NRM5', 'LYO2', 'MMY10', 'NKZ4', 'NLB1',
                 'NLB3', 'NLV1', 'PLC7', 'PLC9', 'PLC4', 'PLW5', 'JSS4',
                 'NFZ13', 'NJO1', 'OEP5', 'OIJ6', 'LWP8', 'MKV10', 'NBX7', 'OHQ5',
                 'OHQ6', 'PIE4', 'NVN2', 'PGL6', "PIT15", 'PIT17', 'PLC6', 'NLV3', 'NPG5'],
         "slow": ['MMY9', 'NGZ5', 'NEL1', 'MMY7', 'NEL3', 'NHR1', 'NPG2', 'NKZ3',
                  'NHR1', 'NLB8', 'NLV2', 'NLV5', 'PLW2', 'OEX5', 'OGV2', 'OEX6',
                  'OEX8', 'LWP2', 'LWP4', 'PLW6', 'PLX13', 'MKV11', 'MKV14',
                  'NGX5', 'ONW8-recheck training', 'OPY5', 'MJL2', 'MJN2', 'JND2',
                  'OHQ3', 'OKK7', 'PIT11', 'MJN3', 'NBE11', 'NCY1', "NIO15", 
                  'OWY6', 'NLN3','LWP6'], 
         "wrong": ['NFZ4', 'NAS9', 'NBX9', 'NJW5', 'MJN5', 'MYC4', 'MZG4','MZG5', 'NCY4',
                   'NCY6', 'NDR1', 'NIO16', "OSA10", 'OSA8', 'OWY5', 'OGY2', 'OKN2',
                   'PGL5', 'PIE8', 'PIT14', 'NEJ1'],
         "nd": ['MYS1', 'NEL5', "NHR7", 'MYS2', 'NRM2', 'NEL2', 'GXS5', 'OSA12']}
def getTiming(x):
    for k in timing.keys():
        if x.name[1] in timing[k]:
            return k
    return 'u'
diffs_by_time = diffs_by_time.set_index(['condition', 'animal','trial no'])
diffs_by_time['learn timing'] = diffs_by_time.groupby(['condition', 'animal','trial no']).apply(getTiming)
diffs_by_time = diffs_by_time.reset_index()
def getLearnType(x):
    for k in types.keys():
        if x.name[1] in types[k]:
            return k
    return 'u'
diffs_by_time = diffs_by_time.set_index(['condition', 'animal','trial no'])
diffs_by_time['learn type'] = diffs_by_time.groupby(['condition', 'animal','trial no']).apply(getLearnType)
diffs_by_time = diffs_by_time.reset_index()


In [ ]:
filt = diffs_by_time[diffs_by_time['diff num'] == 1]
counts_under10 = trial_counts(filt[filt['ITI'] < 10000], ['condition', 'animal', 'Time (hr)', 'diff num'], ['Day', 'Sex', 'Cage', 'ITI','lick', 'acc time'],'trial no')
counts_close  = trial_counts(filt[filt['ITI'] < 3500], ['condition', 'animal', 'Time (hr)', 'diff num'], ['Day', 'Sex', 'Cage', 'ITI','lick', 'acc time'],'trial no')

counts_under10 = counts_under10.sort_values(['condition', 'Time (hr)', 'diff num'])
counts_close = counts_close.sort_values(['condition', 'Time (hr)', 'diff num'])

fracclose = ((counts_close.set_index(['condition', 'animal', 'acc time', 'diff num'])['trial no']) / (counts_under10.set_index(['condition', 'animal', 'acc time', 'diff num'])['trial no']))
fracclose = fracclose.reset_index()
fracclose = fracclose[fracclose['diff num'] == 1]
fracclose = fracclose.set_index(['condition', 'animal']).sort_index()

def getLearnTime(x):
    learntime = fracclose.loc[x.name]
    learntime = learntime[(learntime['acc time'] > 0) & (learntime['trial no'] > 0.3) & (learntime["trial no"].shift(-1) > 0.3)]
    learntime = learntime.groupby(['condition', 'animal'])['acc time'].min()
    if not learntime.empty:
        return learntime.iat[0]
    else:
        return x['acc time'].max()
diffs_by_time = diffs_by_time.set_index(['condition', 'animal'])
diffs_by_time['learn time auto'] = diffs_by_time.groupby(['condition', 'animal']).apply(getLearnTime)
diffs_by_time = diffs_by_time.reset_index()
diffs_by_time = diffs_by_time.sort_values(['condition','Time (hr)', 'learn time auto', 'diff num'])

In [ ]:
#diffs_by_time.to_csv(r'D:\analysis\diffs_by_time.csv', index=False)

In [ ]:
### ITI histograms
pal=sns.color_palette('Spectral', n_colors=19)
diffs_filt = diffs_by_time[(diffs_by_time['condition'].str.contains('DCZ'))]
conds = diffs_filt['condition'].unique()
tims = diffs_filt['acc time'].unique()
tims.sort()
for a in diffs_filt['animal'].unique():
    sns.catplot()


In [ ]:
diffs_filt = diffs_by_time[diffs_by_time['Time (hr)'] <= 48]
diffs_filt = diffs_filt.sort_values(['condition', 'learn time auto', 'animal', 'Time (hr)'])
tims = diffs_filt['Time (hr)'].unique()
tims.sort()
pal=sns.color_palette('Spectral', n_colors=len(tims))
conds = diffs_filt['condition'].unique()
for c in range(len(conds)):
    filts = diffs_filt[diffs_filt['condition'] == conds[c]]
    ans = filts['animal'].unique()
    #ans.sort()
    f = plt.figure(figsize=(5*len(tims),5*len(ans)))
    subfigs = f.subfigures(len(ans), 1)
    print(conds[c], ans)
    for a in range(len(ans)):
        filtsan = diffs_filt[diffs_filt['animal'] == ans[a]]
        if len(ans) > 1:
            ax = subfigs[a].subplots(1, len(tims))
            subfigs[a].suptitle(f'{ans[a]}: {filtsan["learn time auto"].unique()[0] - 24}')
        else:
            ax = subfigs.subplots(1, len(tims))
            subfigs.suptitle(f'{ans[a]}: {filtsan["learn time auto"].unique()[0] - 24}')
        nums = filtsan['diff num'].unique()
        for j in range(1):
            filtsnum = filtsan[filtsan['diff num'] == nums[j]]
            if not filtsnum.empty:
                for n in range(len(tims)):
                    filtstim = filtsnum[filtsnum['Time (hr)'] == tims[n]]
                    if not (filtstim.empty):
                        if j==1:
                            label=ans[a] + " " + str(tims[n])
                        else:
                            label=''
                        # if len(ans) > 1:
                        #     ax[a][n].hist(filtstim['ITI'], color=pal[n],bins=range(2000*(j + 1), 10500*(j + 1), 500),label=label,zorder=len(tims)-n, alpha=0.8)
                        #     ax[a][n].set_xlabel('ITI (ms)')
                        #     ax[a][n].set_ylabel('Frequency')
                        #     ax[a][n].set_title(f'{ans[a]}, {tims[n]}', y=0.7)
                        ax[n].hist(filtstim['ITI'], color=pal[n],bins=range(2000*(j + 1), 10500*(j + 1), 500),label=label,zorder=n, alpha=0.8)
                        ax[n].set_xlabel('ITI (ms)')
                        ax[n].set_ylabel('Frequency')
                        ax[n].set_title(f'{tims[n]}', y=0.9)
        #f.legend(loc='center right', bbox_to_anchor=[1.5, 0.5])
    f.subplots_adjust(hspace=.3, wspace=.3)
    f.suptitle(conds[c],y=1.01)



Totals by condition


In [ ]:
ymax = {0:100, 4:150, 8:500, 12:500,16:500, 20:200, 24:200}
pal=sns.color_palette('Spectral', n_colors=7)
diffs_filt = diffs_by_time[((diffs_by_time['acc time'] <= 24) & (diffs_by_time['acc time'] >= 0))]
#typkeys = list(types.keys())
#typkeys.remove('nd')
tims = diffs_filt['acc time'].unique()
f, ax = plt.subplots(len(conds), len(tims), figsize=(5*len(tims), 5*len(conds)))
for c in range(len(conds)):
    print(conds[c])
    filts = diffs_filt[diffs_filt['condition'] == conds[c]]
    #for i in range(len(typkeys)):
    #    filtstyp  = filts[filts['learn type'] == typkeys[i]] 
    nums = diffs_filt['diff num'].unique()
    #    print(filtstyp['animal'].unique())
    for j in range(1):
            filtsnum = diffs_filt[diffs_filt['diff num'] == nums[j]]
            if not filtsnum.empty:
                for n in range(len(tims)):
                    filtstim = filtsnum[filtsnum['acc time'] == tims[n]]
                    if not (filtstim.empty):
                        # ax[c*(len(typkeys)) + i][n].hist(filtstim['ITI'], color=pal[n],bins=range(2000*(j + 1), 10500*(j + 1), 500*(j + 1)),label=nums[j],zorder=len(tims)-n,alpha=1)
                        # ax[c*(len(typkeys)) + i][n].set_xlabel('ITI (ms)')
                        # ax[c*(len(typkeys)) + i][n].set_ylabel('Frequency')
                        # ax[c*(len(typkeys)) + i][n].set_title(f'{typkeys[i]}, {conds[c]}', y=0.9)
                        ax[n].hist(filtstim['ITI'], color=pal[n],bins=range(2000*(j + 1), 10500*(j + 1), 500),label=label,zorder=n, alpha=0.8)
                        ax[n].set_xlabel('ITI (ms)')
                        ax[n].set_ylabel('Frequency')
                        ax[n].set_title(f'{tims[n]}', y=0.9)
f.subplots_adjust(hspace=.4, wspace=.4)



#### Average by condition


Get bin counts by animal ACC


In [ ]:
histcounts = []
diffs_filt = diffs_by_time#[((diffs_by_time['acc time'] <= 24) & (diffs_by_time['acc time'] >= 0))]
acc = diffs_filt['Day'].unique()
conds = diffs_filt['condition'].unique()
acc.sort()
nums = diffs_filt['diff num'].unique()
for c in range(len(conds)):
    filts = diffs_filt[diffs_filt['condition'] == conds[c]]
    ans = filts['animal'].unique()
    print(ans)
    condhistcounts = []
    condhistind = []
    condhistbin = []
    for a in range(len(ans)):
            filtsan = diffs_filt[diffs_filt['animal'] == ans[a]]
            nums = filtsan['diff num'].unique()
            for j in range(len(nums)):
                filtsnum = filtsan[filtsan['diff num'] == nums[j]]
                if not filtsnum.empty:
                    for n in range(len(acc)):
                        filtstim = filtsnum[filtsnum['Day'] == acc[n]]
                        if not (filtstim.empty):
                            counts, bins = np.histogram(filtstim['ITI'], bins=range(2000*(j + 1), 10500*(j + 1), 200))
                            condhistind.append((conds[c], ans[a], acc[n], nums[j]))
                            condhistcounts.append(counts)
                            condhistbin.append(bins[1:])
    condhistcounts = pd.DataFrame(condhistcounts, index=pd.MultiIndex.from_tuples(condhistind, names=['condition','animal', 'acc time', 'diff num'])).melt(var_name='bin no counts', value_name='ITI', ignore_index=False)
    condhistbin = pd.DataFrame(condhistbin, index=pd.MultiIndex.from_tuples(condhistind,names=['condition', 'animal', 'acc time','diff num'])).melt(var_name='bin no end', value_name='bin end', ignore_index=False)
    condhistcounts = pd.concat([condhistcounts, condhistbin], axis=1) 
    histcounts.append(condhistcounts)
histcountsACC = pd.concat(histcounts)
histcountsACC = histcountsACC.drop(columns='bin no end')

# get mean and SEM
gp = histcountsACC.groupby(['condition', 'acc time', "diff num", 'bin no counts'])
histcountsACC = pd.concat([gp["ITI"].mean().rename("ITI mean"), gp["ITI"].sem().rename("ITI sem"), gp['bin end'].first() - 100], axis=1).reset_index()


Plot ACC


In [ ]:
ymax = {0:8, 4:15, 8:40, 12:40,16:40, 20:25, 24:20}
pal=sns.color_palette('Spectral', n_colors=18)
diffs_filt = histcountsACC[(histcountsACC['condition'].str.contains('ACC')) & ((histcountsACC['acc time'] <= 5))]
diffs_filt = diffs_filt.sort_values('condition')
conds = diffs_filt['condition'].unique()
#typkeys = list(types.keys())
#typkeys.remove('nd')
tims = diffs_filt['acc time'].unique()
tims.sort()
f, ax = plt.subplots(len(conds),len(tims), figsize=(5*len(tims), 5*len(conds)))
for c in range(len(conds)):
    print(conds[c])
    filts = diffs_filt[diffs_filt['condition'] == conds[c]]
 #   for i in range(len(typkeys)):
 #       filtstyp  = filts[filts['learn type'] == typkeys[i]]
    nums = diffs_filt['diff num'].unique()
    for j in range(1):
            filtsnum = filts[filts['diff num'] == nums[j]]
            if not filtsnum.empty:
                for n in range(len(tims)):
                    filtstim = filtsnum[filtsnum['acc time'] == tims[n]]
                    if not (filtstim.empty):
                        l=''
                        if n==0:
                             l=conds[c]
                        ax[c][n].bar(x=filtstim['bin end'], height=filtstim['ITI mean'],
                                    yerr=filtstim['ITI sem'],width=200, color=pal[n],alpha=1,
                                    error_kw={'lw':0.2, 'capsize':0.8,'lolims':True}, zorder=7-n, label=l)
                        ax[c][n].set_xlabel('ITI (ms)')
                        ax[c][n].set_ylabel('Frequency')
                        ax[c][n].set_title(f'{conds[c]} {tims[n]}', y=0.9)
                        ax[c][n].set_ylim([0, 150])
                        ax[c][n].set_xlim([2000,6000])
                    
#f.legend()
f.subplots_adjust(hspace=.3, wspace=.3)
f.suptitle('First 24h',y=0.95)


In [ ]:
ymax = {0:8, 4:15, 8:40, 12:40,16:40, 20:25, 24:20}
pal=sns.color_palette('husl', n_colors=2)
diffs_filt = histcountsACC[(histcountsACC['condition'].str.contains('retrain')) & ((histcountsACC['acc time'] <= 24) & (histcountsACC['acc time'] >= 0))]
diffs_filt = diffs_filt.sort_values(['condition', 'acc time'])

conds = diffs_filt['condition'].unique()
#typkeys = list(types.keys())
#typkeys.remove('nd')
tims = diffs_filt['acc time'].unique()
f, ax = plt.subplots(1,len(tims), figsize=(5*len(tims), 5))
for c in range(len(conds)):
    print(conds[c])
    filts = diffs_filt[diffs_filt['condition'] == conds[c]]
 #   for i in range(len(typkeys)):
 #       filtstyp  = filts[filts['learn type'] == typkeys[i]]
    nums = diffs_filt['diff num'].unique()
    for j in range(1):
            filtsnum = filts[filts['diff num'] == nums[j]]
            if not filtsnum.empty:
                for n in range(len(tims)):
                    filtstim = filtsnum[filtsnum['acc time'] == tims[n]]
                    if not (filtstim.empty):    
                        l=''
                        if n==0:
                             l=conds[c]
                        ax[n].bar(x=filtstim['bin end'], height=filtstim['ITI mean'],
                                    yerr=filtstim['ITI sem'],width=500, color=pal[c],alpha=0.5,label=l,
                                    error_kw={'lw':0.2, 'capsize':0.8,'lolims':True}, zorder=7-n)
                        ax[n].set_xlabel('ITI (ms)')
                        ax[n].set_ylabel('Frequency')
                        ax[n].set_title(f'Time (hr): {tims[n]}', y=0.9)
                        ax[n].set_ylim([0, 125])
                    
f.legend()
f.subplots_adjust(hspace=.3, wspace=.3)
f.suptitle('First 24 hours',y=0.95)


In [ ]:
conds = histcountsACC["condition"].unique()
tcond = histcountsACC["condition"] == 'retrain - train'
rcond = histcountsACC["condition"] == 'retrain - retrain'
filt = (histcountsACC['condition'].str.contains('retrain'))
ind = histcountsACC[filt][["acc time"]].apply(tuple, axis=1).unique()

res=[]
for t in ind:
    daycond = histcountsACC["acc time"] == t
    tr = histcountsACC[filt & daycond & tcond]["ITI mean"]
    rtr = histcountsACC[filt & daycond & rcond]["ITI mean"]
    res.append(stats.ks_2samp(tr, rtr))

pd.DataFrame(res, columns=["statistic", "pvalue"], index=pd.Series(ind).apply(lambda x: 'acc time : ' + str(x[0])))

Get bin counts by animal SAT


In [ ]:
pal=sns.color_palette('Spectral', n_colors=10)
diffs_filt = diffs_by_time[((diffs_by_time['Time (hr)'] >= 0))]
conds = diffs_filt['condition'].unique()
#typkeys = list(types.keys())
#typkeys.remove('nd')
tims = diffs_filt['Time (hr)'].unique()
tims.sort()
nums = diffs_filt['diff num'].unique()
histcounts = []
for c in range(len(conds)):
    filts = diffs_filt[diffs_filt['condition'] == conds[c]]
    #for i in range(len(typkeys)):
    #    filtstyp  = filts[filts['learn type'] == typkeys[i]]
    ans = filts['animal'].unique()
    print(ans)
    condhistcounts = []
    condhistind = []
    condhistbin = []
    for a in range(len(ans)):
            filtsan = diffs_filt[diffs_filt['animal'] == ans[a]]
            nums = filtsan['diff num'].unique()
            for j in range(len(nums)):
                filtsnum = filtsan[filtsan['diff num'] == nums[j]]
                if not filtsnum.empty:
                    for n in range(len(tims)):
                        filtstim = filtsnum[filtsnum['Time (hr)'] == tims[n]]
                        if not (filtstim.empty):
                            counts, bins = np.histogram(filtstim['ITI'], bins=range(2000*(j + 1), 10500*(j + 1), 500))
                            condhistind.append((conds[c],ans[a], tims[n], nums[j]))
                            condhistcounts.append(counts)
                            condhistbin.append(bins[1:])
    condhistcounts = pd.DataFrame(condhistcounts, index=pd.MultiIndex.from_tuples(condhistind, names=['condition','animal', 'Time (hr)', 'diff num'])).melt(var_name='bin no counts', value_name='ITI', ignore_index=False)
    condhistbin = pd.DataFrame(condhistbin, index=pd.MultiIndex.from_tuples(condhistind,names=['condition','animal', 'Time (hr)','diff num'])).melt(var_name='bin no end', value_name='bin end', ignore_index=False)
    condhistcounts = pd.concat([condhistcounts, condhistbin], axis=1) 
    histcounts.append(condhistcounts)
histcountsSAT = pd.concat(histcounts)
histcountsSAT = histcountsSAT.drop(columns='bin no end')

gp = histcountsSAT.groupby(['condition','Time (hr)', "diff num", 'bin no counts'])
histcountsSAT = pd.concat([gp["ITI"].mean().rename("ITI mean"), gp["ITI"].sem().rename("ITI sem"), gp['bin end'].first() - 250], axis=1).reset_index()


Plot SAT


In [ ]:
pal=sns.color_palette('Spectral', n_colors=7)
diffs_filt = histcountsSAT[(histcountsSAT['condition'].str.contains('lesion')) & ((histcountsSAT['Time (hr)'] <= 24) & (histcountsSAT['Time (hr)'] >= 0))]
ymax = {0:8, 4:15, 8:40, 12:40,16:40, 20:25, 24:20}
conds = diffs_filt['condition'].unique()
#typkeys = list(types.keys())
#typkeys.remove('nd')
tims = diffs_filt['Time (hr)'].unique()
f, ax = plt.subplots(len(conds),len(tims), figsize=(5*len(tims), 5*len(conds)))
for c in range(len(conds)):
    print(conds[c])
    filts = diffs_filt[diffs_filt['condition'] == conds[c]]
    #for i in range(len(typkeys)):
    #    filtstyp  = filts[filts['learn type'] == typkeys[i]]
    nums = diffs_filt['diff num'].unique()
    for j in range(1):
            filtsnum = filts[filts['diff num'] == nums[j]]
            if not filtsnum.empty:
                for n in range(len(tims)):
                    filtstim = filtsnum[filtsnum['Time (hr)'] == tims[n]]
                    if not (filtstim.empty):
                        ax[c][n].bar(x=filtstim['bin end'], height=filtstim['ITI mean'],
                                    yerr=filtstim['ITI sem'],width=500, color=pal[n],alpha=1,
                                    error_kw={'lw':0.2, 'capsize':0.8,'lolims':True}, zorder=7-n)
                        ax[c][n].set_xlabel('ITI (ms)')
                        ax[c][n].set_ylabel('Frequency')
                        ax[c][n].set_title(f'{conds[c]}', y=0.9)
                        ax[c][n].set_ylim([0, 125])
                    
#f.legend()
f.subplots_adjust(hspace=.3, wspace=.3)
f.suptitle('SAT24',y=1.01)


In [ ]:
pal=sns.color_palette('husl', n_colors=2)
diffs_filt = histcountsSAT[(histcountsSAT['condition'].str.contains('retrain')) & ((histcountsSAT['Time (hr)'] <= 24) & (histcountsSAT['Time (hr)'] >= 0))]
ymax = {0:8, 4:15, 8:40, 12:40,16:40, 20:25, 24:20}
conds = diffs_filt['condition'].unique()
#typkeys = list(types.keys())
#typkeys.remove('nd')
tims = diffs_filt['Time (hr)'].unique()
f, ax = plt.subplots(1,len(tims), figsize=(5*len(tims), 5))
for c in range(len(conds)):
    print(conds[c])
    filts = diffs_filt[diffs_filt['condition'] == conds[c]]
    #for i in range(len(typkeys)):
    #    filtstyp  = filts[filts['learn type'] == typkeys[i]]
    nums = diffs_filt['diff num'].unique()
    for j in range(1):
            filtsnum = filts[filts['diff num'] == nums[j]]
            if not filtsnum.empty:
                for n in range(len(tims)):
                    filtstim = filtsnum[filtsnum['Time (hr)'] == tims[n]]
                    if not (filtstim.empty):
                        if n==1:
                             l=conds[c]
                        else: 
                            l=''
                        ax[n].bar(x=filtstim['bin end'], height=filtstim['ITI mean'],
                                    yerr=filtstim['ITI sem'],width=500, color=pal[c],alpha=0.5,
                                    error_kw={'lw':0.2, 'capsize':0.8,'lolims':True}, zorder=7-n, label=l)
                        ax[n].set_xlabel('ITI (ms)')
                        ax[n].set_ylabel('Frequency')
                        ax[n].set_title(f'{tims[n]}', y=0.9)
                        ax[n].set_ylim([0, 125])
                    
f.legend()
f.subplots_adjust(hspace=.3, wspace=.3)
f.suptitle('SAT 24',y=1.01)


In [ ]:
conds = histcountsSAT["condition"].unique()
tcond = histcountsSAT["condition"] == 'retrain - train'
rcond = histcountsSAT["condition"] == 'retrain - retrain'
filt = (histcountsSAT['condition'].str.contains('retrain'))
ind = histcountsSAT[filt][["Time (hr)"]].apply(tuple, axis=1).unique()

res=[]
for t in ind:
    daycond = histcountsSAT["Time (hr)"] == t
    tr = histcountsSAT[filt & daycond & tcond]["ITI mean"]
    rtr = histcountsSAT[filt & daycond & rcond]["ITI mean"]
    res.append(stats.ks_2samp(tr, rtr))

pd.DataFrame(res, columns=["statistic", "pvalue"], index=pd.Series(ind).apply(lambda x: 'acc time : ' + str(x[0])))

### Measures of ITI distribution


In [ ]:
ITI_measures = diffs_by_time.set_index(['condition', 'animal', 'trial no'])
#ITI_measures = ITI_measures[(ITI_measures['ITI'] <= 10000)]
ITI_measures = ITI_measures.groupby(['condition', 'learn time auto','animal', 'Day', 'Time (hr)', 'diff num'])['ITI'].apply(lambda x: pd.Series([stats.skew(x), stats.kurtosis(x), x.mean(),x.median(),x.mode().iat[0],x.std(), x.sem()], index=['skew', 'kurtosis', 'mean','median','mode','stdev', 'sem'], name='measures'))
ITI_measures = ITI_measures.reset_index().dropna()
ITI_measures = ITI_measures.rename(columns={'level_6':'measures'})
ITI_measures['acc time'] = ITI_measures.groupby(['condition', 'animal'])["Time (hr)"].transform(lambda x: x - x.iat[0])
ITI_measures["Time (hr)"] = ITI_measures["Time (hr)"] + 2


In [ ]:
colors = sns.color_palette('husl', n_colors=11)
learntimecount = ITI_measures[ITI_measures['condition'].str.contains('retrain')].groupby(['condition', 'animal']).first().reset_index().groupby(['condition', 'learn time auto'])['animal'].count().reset_index()
g = sns.catplot(data=learntimecount,x='condition', y='animal', hue='learn time auto', kind='bar', palette=colors)
g.axes[0][0].set_ylabel('count')

In [ ]:
colors = sns.color_palette('husl', n_colors=2)
#for c in ITI_measures['condition'].unique():
g = sns.relplot(ITI_measures[(ITI_measures['condition'].str.contains('retrain')) & (ITI_measures["Time (hr)"] <=120)],
            x='Time (hr)', y='ITI', kind='line',
            col='measures', col_wrap=3,hue='condition',col_order=['skew', 'kurtosis', 'mean','stdev', 'median', 'mode'], errorbar='se', err_style='bars', err_kws={'lw':0.2}, 
            marker='o', ms='8', mec=None, lw=1, palette=['lightblue','mediumblue', 'black'],facet_kws={'sharey':False})
for(name, ax) in g.axes_dict.items():
        ax.set_title(name)
        ax.set_xlim([-24,48])
        ax.xaxis.set_major_locator(ticker.MultipleLocator(24))
        ax.xaxis.set_minor_locator(ticker.MultipleLocator(6))
        #ax.set_xticklabels(ax.get_xticklabels(), rotation=90, fontsize=18)
        
        if name in ['mean']:
            ax.set_ylim([0, 150000])
        if name in ['mode', 'median']:
             ax.set_ylim([0,10000])
        if name == 'stdev':
            ax.set_ylim([0, 2000000])
g.fig.suptitle('ITI measures', y=1.05)


In [ ]:
diffs_filt = diffs_by_time[(diffs_by_time['ITI'] <= 10000)]
diffs_filt[(diffs_filt['Day'] == 4) & (diffs_filt['condition'] == 'Contra-lesion')]

In [ ]:
first20 = get_nth_part_day(diffs_by_time, 5,1)
first20measures = first20.set_index(['condition', 'animal', 'trial no'])
first20measures = first20measures.groupby(['condition','animal', 'Day', 'diff num'])['ITI'].apply(lambda x: pd.Series([stats.skew(x), stats.kurtosis(x), x.mean(),x.median(),x.mode().iat[0],x.std(), x.sem()], index=['skew', 'kurtosis', 'mean','median','mode','stdev', 'sem'], name='measures'))
first20measures = first20measures.reset_index()#.dropna()
first20measures = first20measures.rename(columns={'level_4':'measures'})


In [ ]:
last20 = get_nth_part_day(diffs_by_time, 5,5)
last20measures = last20.set_index(['condition', 'animal', 'trial no'])
last20measures = last20measures.groupby(['condition','animal', 'Day', 'diff num'])['ITI'].apply(lambda x: pd.Series([stats.skew(x), stats.kurtosis(x), x.mean(),x.median(),x.mode().iat[0],x.std(), x.sem()], index=['skew', 'kurtosis', 'mean','median','mode','stdev', 'sem'], name='measures'))
last20measures = last20measures.reset_index().dropna()
last20measures = last20measures.rename(columns={'level_5':'measures'})


In [ ]:
first20measures['fraction'] = 'first 20%'
last20measures['fraction'] = 'last 20%'
nthmeasures = pd.concat([first20measures, last20measures])
nthmeasures['fracday'] = nthmeasures['Day'].astype(str) + ' ' + nthmeasures['fraction']
nthmeasures = nthmeasures.sort_values(['condition',  'Day', 'fraction'])


In [ ]:
#condfilt = nthmeasures[((nthmeasures['condition'] == 'SAT') | (nthmeasures['condition'] == 'all_reward') | (nthmeasures['condition'].str.contains('cno')) & (nthmeasures['diff num'] == 1))]
condfilt = nthmeasures[(nthmeasures['condition'].str.contains('lesion'))]
condfilt = condfilt[(condfilt['diff num'] == 1)]
condfilt = condfilt.reset_index(drop=True)
g = sns.catplot(condfilt[(condfilt['fracday'].str.contains('last')) & (condfilt['Day'] < 6)], 
                y="ITI",row='fracday', x="fracday", hue='condition',kind='bar', hue_order=['Contra-lesion','Ipsi-lesion'],#hue_order=['fast', 'med','slow', 'wrong'],
                col="level_4", sharey=False, sharex=False, legend=True, palette='husl')
for (n,ax) in g.axes_dict.items():
    ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right')
    ax.set_title(n)
    if n[1] in ['median', 'mode']:
        ax.set_ylim([0, 25000])
    if n[1] == 'mean':
        ax.set_ylim([0, 200000])
    if n[1] == 'skew':
        ax.set_ylim([0, 15])
    if n[1] == 'kurtosis':
        ax.set_ylim([-1, 200])
    if n[1] == 'stdev':
        ax.set_ylim([0, 2000000])
    if n[1] == 'sem':
        ax.set_ylim([0,200000])
g.fig.subplots_adjust(hspace=0.8)
g = sns.catplot(condfilt, y="ITI",x="fracday", row='condition', hue='fracday',kind='bar',
                            col="level_4", sharey=False, sharex=False, legend=True, palette='Spectral')


Trial counts by learn type


In [ ]:
kp = ['Day', 'Sex', 'Cage', 'ITI', 'lick', 'learn time auto', 'acc time', 'Strain']
counts = trial_counts(diffs_by_time, ['condition', 'animal', 'Time (hr)', 'diff num'], kp,'trial no')
counts_under10 = trial_counts(diffs_by_time[diffs_by_time['ITI'] < 10000],  ['condition', 'animal', 'Time (hr)', 'diff num'],kp,'trial no')
counts_close  = trial_counts(diffs_by_time[diffs_by_time['ITI'] < 3500], ['condition', 'animal', 'Time (hr)', 'diff num'], kp,'trial no')

counts = counts.sort_values(['condition', 'Time (hr)', 'diff num', 'learn time auto'])
counts_under10 = counts_under10.sort_values(['condition', 'Time (hr)', 'diff num', 'learn time auto'])
counts_close = counts_close.sort_values(['condition', 'Time (hr)', 'diff num', 'learn time auto'])
kps = counts_close.groupby(['condition', 'animal', 'Time (hr)', 'diff num', 'learn time auto']).first()
fracclose = ((counts_close.set_index(['condition', 'animal', 'Time (hr)', 'diff num', 'learn time auto'])['trial no']) / (counts.set_index(['condition', 'animal', 'Time (hr)', 'diff num', 'learn time auto'])['trial no']))
fracclose = fracclose.rename('frac')
fracclose = pd.concat([fracclose, kps], axis=1)
fracclose = fracclose.reset_index()
fracclose = fracclose[fracclose['diff num'] == 1]
fracclose = fracclose.dropna()


In [ ]:
g = plot_hr(counts[(counts['diff num'] == 1) & (counts['condition'].str.contains('retrain'))], y='trial no', hue='condition',
            #hue_order=[4,8,12,16,20,24,28,32,36,40,44,48,'wrong'],col_wrap=4,
            ylim=[0, 800],xlim=[-24, 48],
            errorbar='se', err_style='bars', err_kws={'lw':1,'capsize':0.8}, palette='husl',legend='full')
for (name, ax) in g.axes_dict.items():
    if name == 'SAT' or name == 'SATnew' or name == 'all_reward' or 'cno' in name:
        ax.set_xlim([-24, 24])
    if name=='acc3':
        ax.set_ylim([0, 600])
g.fig.suptitle('all trials', y=1.01, fontsize=20)


In [ ]:
g = plot_hr(counts_under10[(counts_under10['diff num'] == 1)  & (counts_under10['condition'].str.contains('lesion'))], y='trial no', hue='condition', 
            #hue_order=[4,8,12,16,20,24,28,32,36,40,44,48,'wrong','u'],col_wrap=4,
              ylim=[0, 800],palette='husl',xlim=[-24, 48],ymajmult=200, yminmult=100,
            errorbar='se', err_style='bars', err_kws={'lw':1,'capsize':0.8}, legend='full')
for (name, ax) in g.axes_dict.items():
    if name == 'SAT' or name == 'SATnew' or name == 'all_reward' or 'cno' in name:
        ax.set_xlim([-24, 24])
    if name=='acc3':
        ax.set_ylim([0, 600])
g.fig.suptitle('Trials with ITI <= 10s',y=1.01, fontsize=20)


In [ ]:
fracunder10 = ((counts_under10.set_index(['condition', 'animal', 'Time (hr)', 'diff num', 'learn time auto'])['trial no']) / (counts.set_index(['condition', 'animal', 'Time (hr)', 'diff num', 'learn time auto'])['trial no']))
fracunder10 = fracunder10.reset_index()
fracunder10 = fracunder10[fracunder10['diff num'] == 1]
fracunder10 = fracunder10.dropna()


In [ ]:
fracunder10['Time (hr)'] = fracunder10['Time (hr)'] + 2

In [ ]:
g = plot_hr(fracunder10[fracunder10['condition'].str.contains('retrain')], y='trial no', col=None, hue='condition',ymajmult=0.5, yminmult=0.1,
            #hue_order=[4,8,12,16,20,24,28,32,36,40,44,48,'wrong'],col_wrap=4,
              ylim=[0, 1],xlim=[-24, 48],
            errorbar='se', palette=['lightblue','mediumblue', ],legend='full',err_style='bars', err_kws={'lw':1,'capsize':0.8})
for (name, ax) in g.axes_dict.items():
    if name == 'SAT' or name == 'SATnew' or name == 'all_reward' or 'cno' in name:
        ax.set_xlim([-24, 24])
g.figure.suptitle('fraction of trials with ITI <=10s', y=1.01, fontsize=20)
g.axes[0][0].axhline(y=0.75, xmin=0, xmax=1, ls="-", color="black", zorder=10, lw=1)


In [ ]:
fracclose = ((counts_close.set_index(['condition', 'animal', 'acc time', 'diff num', 'learn time auto'])['trial no']) / (counts.set_index(['condition', 'animal', 'acc time', 'diff num', 'learn time auto'])['trial no']))
fracclose = fracclose.reset_index()
fracclose = fracclose[fracclose['diff num'] == 1]
fracclose = fracclose.dropna()

In [ ]:
fracclose['Time (hr)'] = fracclose["Time (hr)"] + 2

In [ ]:
g = plot_hr(fracclose[(fracclose['condition'].str.contains('retrain'))]
            , ymajmult=0.5, yminmult=0.1,
            x='Time (hr)',y='frac', col=None, hue='condition', 
            #hue_order=[4,8,12,16,20,24,28,32,36,40,44,48,'wrong'],
             ylim=[0, 1.05], palette=['lightblue','mediumblue', ], legend='full',xlim=[-24, 48],
            errorbar='se', err_style='bars', err_kws={'lw':1,'capsize':0.8})
for (name, ax) in g.axes_dict.items():
    #if name == 'SAT' or name == 'SATnew' or name == 'all_reward' or 'cno' in name:
    #    ax.set_xlim([0, 24])
    #ax.axhline(y=0.4, xmin=0, xmax=1, ls="-", color="black", zorder=10, lw=1)
    ax.axhline(y=0.35, xmin=0, xmax=1, ls="-", color="blue", zorder=10, lw=1)
    #ax.axhline(y=0.3, xmin=0, xmax=1, ls="-", color="black", zorder=10, lw=1)
#g.axes[0][0].axhline(y=0.35, xmin=0, xmax=1, ls="-", color="blue", zorder=10, lw=1)
g.axes[0][0].axhline(y=0.5, xmin=0, xmax=1, ls="-", color="black", zorder=10, lw=1)
g.figure.suptitle('fraction of trials within 1 second of min ITI', y=1.01, fontsize=20)


### Performance for trials filtered by ITI <= 10s


In [ ]:
def get_ITI(trial, df):
    (cond, an, trial_no, l) = trial
    try:
        return pd.Series([df.loc[(cond, an, trial_no)] for i in range(l)])
    except:
        return pd.Series([pd.NA for i in range(l)])

def labelITI(data, i, diffs):
    iti = diffs[(diffs['diff num'] == i)].set_index(['condition', 'animal', 'trial no'])['ITI']
    iti = iti.sort_index()

    num_samples = data.groupby(['condition', 'animal', 'trial no']).count()['lick']
    iti_count = pd.Series(num_samples.reset_index().itertuples(index=False,name=None))

    # # label all samples in a trial with its trial number
    iti_count = iti_count.map(lambda x: get_ITI(x, iti)).explode()
    iti_count.index = data.index

    # # add trial labels to data
    data[i] = iti_count

data = df.set_index(['condition', 'animal', 'timestamp'])
labelITI(data, 1, diffs_by_time)
data = data.rename(columns={1:'ITI'})
data = data.reset_index()
#data = data.melt(id_vars=kpcols,value_vars = range(1,2), var_name='diff num', value_name='ITI',ignore_index=False)


In [ ]:
under10 = data[data['ITI'] <= 10000]
over10 = data[data['ITI'] > 10000]

Lick frequency analysis


In [ ]:
keep.remove('ITI')
keep.remove('diff num')

In [ ]:
freq_window = None # milliseconds, length of rolling window
freq_bin = 100 # milliseconds, length of discrete time bin
keys["millis bin"] = "puff delta"
time_bin = 4*60 # minutes

data = full_lickfreq(df, freq_window, freq_bin, time_bin, keys, keep, values)

flkeep = keep + ["Time (hr)", "delta", "Day", "day_delta", "puff delta", "Time (ms)"]

keys["time bin"] = "delta"
means, flcounts, perf = agg(data, keys, index, flkeep, values)
daytots = sum_trials(flcounts, index + ["Day"],[], "trial no")
hourtots = sum_trials(flcounts, index + ["Time (hr)"],[], "trial no")

mins["min_trials"] = 10 # minimum number of trials in a bin to keep
mins["min_blank"] = 1 # minimum number of water trials
mins["min_stimulus"] = 1 # minimum number of water trials

data_filtered = drop_group(data, mins, keys, "trial no", index, flkeep)
filtmeans, flfiltcounts, filtperf = agg(data_filtered, keys, index, flkeep, values)
daytotsfilt = sum_trials(flfiltcounts, index + ["Day"],[], "trial no")
hourtotsfilt = sum_trials(flfiltcounts, index + ["Time (hr)"], [],"trial no")

In [ ]:
# means.to_csv(rf'D:\analysis\means{freq_bin}_{time_bin}.csv', index=False)
# perf.to_csv(rf'D:\analysis\perf{freq_bin}_{time_bin}.csv', index=False)

# filtmeans.to_csv(rf'D:\analysis\filtmeans_{freq_bin}_{time_bin}_{mins["min_trials"]}_{mins["min_blank"]}_{mins["min_stimulus"]}.csv', index=False)
# filtperf.to_csv(rf'D:\analysis\filtperf{freq_bin}_{time_bin}_{mins["min_trials"]}_{mins["min_blank"]}_{mins["min_stimulus"]}.csv', index=False)


In [ ]:
for c in filtmeans['condition'].unique():
    #if 'lesion test' in c:
        for a in filtmeans[(filtmeans['condition'] == c)]['animal'].unique():
                plot_lickfreq(filtmeans[(filtmeans['animal'] == a)], suptitle=f'{c}: {a}',
                            col_wrap=6,errorbar='se', err_style='bars', err_kws={'lw':0.7})

In [ ]:
for c in filtmeans['condition'].unique():
    #if 'control' in c:
        plot_lickfreq(filtmeans[(filtmeans['condition'] == c) 
                                &((filtmeans['Time (hr)'] == 4)) #| (filtmeans['Time (hr)'] == 4) | (filtmeans['Time (hr)'] == 44)) 
                                ], 
                                suptitle=c, col_wrap=1,errorbar='se', err_style='bars', err_kws={'lw':0.7}, xlim=[-200, 1000])

In [ ]:
time_bin = 4*60 # minutes
r = (700, 1000)
keys["millis bin"] = None
ant_data = bin_lickfreq(df, r, time_bin, keys, keep, values)

antkeep = keep + ["Time (hr)", "delta", "Day", "day_delta"]
keys["time bin"] = "delta"
keys["millis bin"] = None
ant_means, counts, ant_perf  = agg(ant_data, keys, index, antkeep, values)
daytots = sum_trials(counts, index + ["Day"], [],"trial no")
hourtots = sum_trials(counts, index + ["Time (hr)"], [],"trial no")

mins["min_trials"] = 10 # minimum number of trials in a bin to keep
mins["min_blank"] = 1 # minimum number of water trials
mins["min_stimulus"] = 1 # minimum number of water trials
keys["time bin"] = "delta"
keys["millis bin"] = None

ant_data_filtered = drop_group(ant_data, mins, keys, "trial no", index, antkeep)
antfiltmeans, filtcounts, antfiltperf = agg(ant_data_filtered, keys, index, antkeep, values)
daytotsfilt = sum_trials(filtcounts, index + ["Day"], ['cond'],"trial no")
hourtotsfilt = sum_trials(filtcounts, index + ["Time (hr)"], ['cond'],"trial no")

parts = 5 # number of partitions to divide each day's trials into 
nth_part = 5 # which partition of trials to get (1-indexed)
nth_part_data = get_nth_part_day(ant_data, parts, nth_part)

keys["time bin"] = "day_delta"
keys['millis bin'] = None
nthpart_lickfreq, nthpart_counts, nthpart_perf = agg(nth_part_data, keys, index, antkeep, values)

nthpart_lickfreq = nthpart_lickfreq.set_index(['animal', 'Day'])
nthpart_lickfreq = nthpart_lickfreq[nthpart_lickfreq.groupby(['animal', 'Day']).count()['lick'] == 2]
nthpart_lickfreq = nthpart_lickfreq.reset_index()

In [ ]:
last20 = get_nth_part_day(ant_data_filtered, 5, 5)
first20 = get_nth_part_day(ant_data_filtered, 5, 1)

keys["time bin"] = "day_delta"
keys['millis bin'] = None
last20_lickfreq, last20_counts, last20_perf = agg(last20, keys, index, antkeep, values)
first20_lickfreq, first20_counts, first20_perf = agg(first20, keys, index, antkeep, values)

last20_lickfreq = last20_lickfreq.set_index(['animal', 'Day'])
last20_lickfreq = last20_lickfreq[last20_lickfreq.groupby(['animal', 'Day']).count()['lick'] == 2]
last20_lickfreq = last20_lickfreq.reset_index()

first20_lickfreq = first20_lickfreq.set_index(['animal', 'Day'])
first20_lickfreq = first20_lickfreq[first20_lickfreq.groupby(['animal', 'Day']).count()['lick'] == 2]
first20_lickfreq = first20_lickfreq.reset_index()
first20_lickfreq['time'] = 'first 20%'
last20_lickfreq['time'] = 'last 20%'

first20_perf['time'] = 'first 20%'
last20_perf['time'] = 'last 20%'

firstlast20_perf = pd.concat([first20_perf, last20_perf])
firstlast20_perf = firstlast20_perf[firstlast20_perf['Time (hr)'] < 48]
firstlast20_lickfreq = pd.concat([first20_lickfreq, last20_lickfreq])
firstlast20_lickfreq = firstlast20_lickfreq[firstlast20_lickfreq['Time (hr)'] < 48]

In [ ]:
firstlast20_lickfreq.loc[~(firstlast20_lickfreq['condition'].str.contains('test')) 
                   & ~(firstlast20_lickfreq['condition'].str.contains('retrain')),'Day_c'] = firstlast20_lickfreq.loc[~(firstlast20_lickfreq['condition'].str.contains('test')) 
                                                                                                            & ~(firstlast20_lickfreq['condition'].str.contains('retrain')),'Day']
firstlast20_lickfreq.loc[firstlast20_lickfreq['condition'].str.contains('retrain'),'Day_c'] = firstlast20_lickfreq.loc[firstlast20_lickfreq['condition'].str.contains('retrain'),'Day'] + 2
firstlast20_lickfreq.loc[firstlast20_lickfreq['condition'].str.contains('test'),'Day_c'] = firstlast20_lickfreq.loc[firstlast20_lickfreq['condition'].str.contains('test'),'Day'] + 4
minday = int(firstlast20_lickfreq["Day_c"].min())
firstlast20_lickfreq["daytime"] = firstlast20_lickfreq['time'] + ' ' + firstlast20_lickfreq["Day_c"].apply(lambda x: day_to_label(minday, x))
firstlast20_lickfreq.loc[firstlast20_lickfreq['time'] == 'last 20%','Day_c'] = firstlast20_lickfreq.loc[firstlast20_lickfreq['time'] == 'last 20%','Day_c'] + 0.5

firstlast20_lickfreq.loc[firstlast20_lickfreq['animal'].str.contains('train')
                       | firstlast20_lickfreq['animal'].str.contains('test')
                       | firstlast20_lickfreq['animal'].str.contains('test'),'animal'] = firstlast20_lickfreq.loc[
                                                                                           firstlast20_lickfreq['animal'].str.contains('train')
                                                                                         | firstlast20_lickfreq['animal'].str.contains('test')
                                                                                         | firstlast20_lickfreq['animal'].str.contains('test'),'animal'].str.rstrip('retains ')
firstlast20_lickfreq['condition'] = firstlast20_lickfreq['condition'].str.rstrip('retains ')
firstlast20_lickfreq.loc[firstlast20_lickfreq['condition'].str.contains('lesio'),'condition'] = firstlast20_lickfreq.loc[firstlast20_lickfreq['condition'].str.contains('lesio'),'condition'] + 'n'
minday = int(firstlast20_lickfreq["Day"].min())
firstlast20_lickfreq["Day"] = firstlast20_lickfreq["Day"].apply(lambda x: day_to_label(minday, x))
firstlast20_lickfreq['daytimecond'] = firstlast20_lickfreq['condition'] + ' ' + firstlast20_lickfreq['daytime']

firstlast20_lickfreq = firstlast20_lickfreq.dropna()

In [ ]:
firstlast20_perf.loc[~(firstlast20_perf['condition'].str.contains('test')) 
                   & ~(firstlast20_perf['condition'].str.contains('retrain')),'Day_c'] = firstlast20_perf.loc[~(firstlast20_perf['condition'].str.contains('test')) 
                                                                                                            & ~(firstlast20_perf['condition'].str.contains('retrain')),'Day']
firstlast20_perf.loc[firstlast20_perf['condition'].str.contains('retrain'),'Day_c'] = firstlast20_perf.loc[firstlast20_perf['condition'].str.contains('retrain'),'Day'] + 2
firstlast20_perf.loc[firstlast20_perf['condition'].str.contains('test'),'Day_c'] = firstlast20_perf.loc[firstlast20_perf['condition'].str.contains('test'),'Day'] + 4
minday = int(firstlast20_perf["Day_c"].min())
firstlast20_perf["daytime"] = firstlast20_perf['time'] + ' ' + firstlast20_perf["Day_c"].apply(lambda x: day_to_label(minday, x))
firstlast20_perf.loc[firstlast20_perf['time'] == 'last 20%','Day_c'] = firstlast20_perf.loc[firstlast20_perf['time'] == 'last 20%','Day_c'] + 0.5

firstlast20_perf.loc[firstlast20_perf['animal'].str.contains('train')
                       | firstlast20_perf['animal'].str.contains('test')
                       | firstlast20_perf['animal'].str.contains('test'),'animal'] = firstlast20_perf.loc[
                                                                                           firstlast20_perf['animal'].str.contains('train')
                                                                                         | firstlast20_perf['animal'].str.contains('test')
                                                                                         | firstlast20_perf['animal'].str.contains('test'),'animal'].str.rstrip('retains ')
firstlast20_perf['condition'] = firstlast20_perf['condition'].str.rstrip('retains ')
firstlast20_perf.loc[firstlast20_perf['condition'].str.contains('lesio'),'condition'] = firstlast20_perf.loc[firstlast20_perf['condition'].str.contains('lesio'),'condition'] + 'n'
minday = int(firstlast20_perf["Day"].min())
firstlast20_perf["Day"] = firstlast20_perf["Day"].apply(lambda x: day_to_label(minday, x))
firstlast20_perf['daytimecond'] = firstlast20_perf['cond'] + ' ' + firstlast20_perf['daytime']

firstlast20_perf = firstlast20_perf.dropna()

In [ ]:
firstlast20_perf[firstlast20_perf['animal'] == 'PB715']

In [ ]:
CO = ['control', 'sham lesion', 'ipsi lesion', 'contra lesion']#,'SAT5','Contra-lesion', 'Ipsi-lesion']#, 'sham lesion']
o = ['last 20% ACC 0', 'last 20% SAT 2','first 20% SAT 3', "last 20% SAT 4"]
tmp = firstlast20_perf[firstlast20_perf.apply(lambda x: x["daytime"] in o, axis=1) & firstlast20_perf.apply(lambda x: x["condition"] in CO, axis=1)]

g = sns.catplot(tmp, x='condition', y='lick', hue='daytime',kind='point', linestyle='none',order=CO,palette='rocket',
                errorbar='se',err_kws={'lw':1, 'zorder':0}, markersize=10, hue_order=o,
                dodge=0.5, capsize=0.05, alpha=0.3, aspect=1.5, zorder=0.1, legend=True)
sns.stripplot(tmp, x="condition", y="lick", hue='daytime',size=5, dodge=True, hue_order=o,
              legend=False, jitter=False, alpha=1,order=CO,palette='rocket',
              )
segs1 = []
for i in range(len(CO)):
    cdfilt = tmp[tmp['condition'] ==  CO[i]]
    for a in cdfilt["animal"].unique():
        anfilt = cdfilt[cdfilt["animal"] == a]
        pts = []
        for j in range(len(o)):
            op = list(anfilt[(anfilt["daytime"] == o[j])]["lick"])
            if (len(op) != 0):
                pts.append([i + 1/(len(o)+1)*(j-(len(o)-1)/2), op[0]])
        if(len(pts) != 0):
            segs1.append(pts)
linesegs = LineCollection(segs1,color='k', lw=0.5, zorder=1)
g.axes[0][0].add_collection(linesegs)

g.axes[0][0].set_xticklabels(g.axes[0][0].get_xticklabels(), rotation=45, ha='right')
g.axes[0][0].set_ylim([-10, 10])
g.axes[0][0].set_ylabel('Performance')
g.axes[0][0].set_xlabel('Condition')

In [ ]:
res = []
firstlast20_perf = firstlast20_perf.sort_values(['condition', 'animal', 'day_delta'])

cond1 = firstlast20_perf["daytime"] == "retrain last 20% SAT 2"
cond2 = firstlast20_perf["daytime"] == "retrain first 20% SAT 1"

filt = (firstlast20_perf['condition'].str.contains('control')) | (firstlast20_perf['condition'] == ('contra lesion'))
filt = filt & ~(firstlast20_perf['animal'].str.contains('PB'))
filt =  filt & ((firstlast20_perf["Day"] == 'ACC 0') | (firstlast20_perf["Day"] == 'SAT 2') | (firstlast20_perf["Day"] == 'SAT 1'))
#ind = firstlast20_perf[filt][["Day", "condition"]].apply(tuple, axis=1).unique()
ind = firstlast20_perf[filt]['condition'].unique()

for c in ind:
    ccond = firstlast20_perf["condition"] == c
    filt1 = firstlast20_perf[filt & cond1 & ccond]["lick"]
    san = firstlast20_perf[filt & cond1  & ccond][['animal',"lick"]]
    filt2 = firstlast20_perf[filt & cond2 & ccond]["lick"]
    ban = firstlast20_perf[filt & cond2  & ccond][['animal',"lick"]]
    #print(c,'train', san,'retrain', ban, '\n')
    res.append(stats.wilcoxon(filt1, filt2))

pd.DataFrame(res, columns=["statistic", "pvalue"], index=ind)#pd.Series(ind).apply(lambda x: x[1] + ", " + x[0]))

In [ ]:
tmp = firstlast20_lickfreq[firstlast20_lickfreq.apply(lambda x: x['condition'] in CO, axis=1) & (firstlast20_lickfreq.apply(lambda x: x['daytime'] in o, axis=1)) 
                           ]
g = sns.catplot(tmp, y='lick', x='daytime',kind='bar', hue='stimulus', col='condition',order=o,errorbar='se',err_kws={'lw':1, 'zorder':0}, alpha=0.3, palette=['red', 'green'], 
                hue_order=['blank', 'stimulus'], col_order=CO)
for name, ax in g.axes_dict.items():
    g.axes_dict[name].set_title(name)
    g.axes_dict[name].set_xticklabels(g.axes_dict[name].get_xticklabels(), rotation=45, ha='right')
    sns.stripplot(tmp[tmp['condition'] == name], ax=g.axes_dict[name],x="daytime", y="lick",hue='stimulus',size=5, dodge=True, legend=False, palette=['red', 'green'],
                   hue_order=['blank', 'stimulus'], order=o, jitter=False)
    

for i in range(len(CO)):
    segs=[]
    print(CO[i])
    cdfilt = tmp[tmp['condition'] ==  CO[i]]
    for a in cdfilt["animal"].unique():
        anfilt = cdfilt[cdfilt["animal"] == a]
        for j in range(len(o)):        
            pts = []
            for stim in ['stimulus', 'blank']:
                c = (anfilt["daytime"] == o[j]) & (anfilt['stimulus'] == stim)
                op = list(anfilt[c]["lick"])
                if (len(op) != 0):
                    if stim =='stimulus':
                        s = 0.25
                    else:
                        s = -0.25
                    pts.append([j + s, op[0]])
            if(len(pts) != 0):
                segs.append(pts)
    linesegs = LineCollection(segs,color='k', lw=0.5, zorder=1)
    g.axes_dict[CO[i]].add_collection(linesegs)

g.axes[0][0].set_ylabel('Lick freq')
g.axes[0][0].set_ylim([0,10])

In [ ]:
res = []
firstlast20_lickfreq = firstlast20_lickfreq.sort_values(['condition', 'animal', 'day_delta'])

cond1 = firstlast20_lickfreq["condition"] == "Contra-lesion"
cond2 = firstlast20_lickfreq["condition"] == "Ipsi-lesion"

filt = (firstlast20_lickfreq.apply(lambda x: x['Day'] in o, axis=1)) & (firstlast20_lickfreq['time'] == 'last 20%')
#filt = filt & ~(firstlast20_perf['animal'].str.contains('PB'))
#filt =  filt & ((firstlast20_perf["Day"] == 'ACC 0') | (firstlast20_lickfreq["Day"] == 'SAT 2') | (firstlast20_lickfreq["Day"] == 'SAT 1'))
ind = firstlast20_lickfreq[filt][["Day", 'time', 'stimulus']].apply(tuple, axis=1).unique()


for t,tt, s in ind:
    #ccond = firstlast20_perf["condition"] == c
    tcond = firstlast20_lickfreq['Day'] == t
    ttcond = firstlast20_lickfreq['time'] == tt
    scond = firstlast20_lickfreq['stimulus'] == s
    filt1 = firstlast20_lickfreq[filt & tcond &ttcond& cond1 & scond]["lick"]
    filt2 = firstlast20_lickfreq[filt & cond2 & scond & tcond&ttcond]["lick"]
    if len(filt1) > 0 and len(filt2) > 0:
        res.append(stats.mannwhitneyu(filt1, filt2))
    else:
        res.append(pd.NA)

pd.DataFrame(res, columns=["statistic", "pvalue"], index=pd.Series(ind).apply(lambda x: x[0] + ", " + x[1] + ', ' + x[2]))

In [ ]:
tmp = (firstlast20_lickfreq[(firstlast20_lickfreq['daytime'] == o[1])].set_index(['condition', 'animal', 'stimulus'])['lick'] -  firstlast20_lickfreq[(firstlast20_lickfreq['daytime'] == o[0])].set_index(['condition', 'animal', 'stimulus'])['lick']).reset_index()
g = sns.catplot(tmp, y='lick', x='condition',kind='bar', hue='stimulus', errorbar='se',err_kws={'lw':1, 'zorder':0}, alpha=0.3, palette=['red', 'green'], hue_order=['blank', 'stimulus'], order=CO)
sns.swarmplot(tmp, x="condition", y="lick", hue='stimulus',size=5, dodge=True, legend=False, palette=['red', 'green'], hue_order=['blank', 'stimulus'], order=CO)
g.axes[0][0].set_ylabel('Lick freq diff\n(test start - train end)')
g.axes[0][0].set_xticklabels(g.axes[0][0].get_xticklabels(), rotation=45, ha='right')
g.axes[0][0].set_ylim([-5,5])

In [ ]:
tmp = (firstlast20_lickfreq[(firstlast20_lickfreq['daytime'] == 'first 20% SAT 3')].set_index(['condition', 'animal', 'stimulus'])['lick'] -  firstlast20_lickfreq[(firstlast20_lickfreq['daytime'] == o[0])].set_index(['condition', 'animal', 'stimulus'])['lick']).reset_index()
g = sns.catplot(tmp, y='lick', x='stimulus',kind='bar', hue='condition', errorbar='se',err_kws={'lw':1, 'zorder':0}, alpha=0.3, #palette=['red', 'green'],
                 order=['blank', 'stimulus'], hue_order=CO)
sns.swarmplot(tmp, x="stimulus", y="lick", hue='condition',size=5, dodge=True, legend=False, #palette=['red', 'green'], 
              order=['blank', 'stimulus'], hue_order=CO)
g.axes[0][0].set_ylabel('Lick freq diff\n(retrain start - ACC end)')
g.axes[0][0].set_xticklabels(g.axes[0][0].get_xticklabels(), rotation=45)
g.axes[0][0].set_ylim([-5,5])

In [ ]:
firstlast20_lickfreq['cond'].unique()

In [ ]:
o

In [ ]:
CO = ['SAT5', 'Contra-lesion', 'Ipsi-lesion']
CO = ['control train', 'sham lesion train', 'ipsi lesion train', 'contra lesion train']
#CO = ['control train', 'control retrain', 'control test', 'contra lesion train', 'contra lesion retrain', 'contra lesion test']
tmp = (firstlast20_lickfreq[(firstlast20_lickfreq['daytime'] == o[1])].set_index(['cond', 'animal', 'stimulus'])['lick'] -  firstlast20_lickfreq[(firstlast20_lickfreq['daytime'] == o[0])].set_index(['cond', 'animal', 'stimulus'])['lick']).reset_index()
g = sns.catplot(tmp, y='lick', x='stimulus',kind='bar', hue='cond', errorbar='se',err_kws={'lw':1, 'zorder':0}, alpha=0.3, #palette=['red', 'green'],
                 order=['blank', 'stimulus'], hue_order=CO)
sns.swarmplot(tmp, x="stimulus", y="lick", hue='cond',size=5, dodge=True, legend=False, #palette=['red', 'green'], 
              order=['blank', 'stimulus'], hue_order=CO)
g.axes[0][0].set_ylabel('Lick freq diff\n(SAT 1 end - ACC end)')
g.axes[0][0].set_xticklabels(g.axes[0][0].get_xticklabels(), rotation=45)
g.axes[0][0].set_ylim([-5,5])

In [ ]:
tmp = (firstlast20_lickfreq[(firstlast20_lickfreq['daytime'] == o[2])].set_index(['cond', 'animal', 'stimulus'])['lick'] -  firstlast20_lickfreq[(firstlast20_lickfreq['daytime'] == o[0])].set_index(['cond', 'animal', 'stimulus'])['lick']).reset_index()
g = sns.catplot(tmp, y='lick', x='stimulus',kind='bar', hue='cond', errorbar='se',err_kws={'lw':1, 'zorder':0}, alpha=0.3, #palette=['red', 'green'],
                 order=['blank', 'stimulus'], hue_order=CO)
sns.swarmplot(tmp, x="stimulus", y="lick", hue='cond',size=5, dodge=True, legend=False, #palette=['red', 'green'], 
              order=['blank', 'stimulus'], hue_order=CO)
g.axes[0][0].set_ylabel('Lick freq diff\n(SAT 2 end - ACC end)')
g.axes[0][0].set_xticklabels(g.axes[0][0].get_xticklabels(), rotation=45)
g.axes[0][0].set_ylim([-5,5])

In [ ]:
tmpdiffs = []
c  = firstlast20_lickfreq.apply(lambda x: x['condition'] in CO, axis=1)
acc = firstlast20_lickfreq[c & (firstlast20_lickfreq['daytime'] == 'last 20% ACC 0')].set_index(['condition', 'animal', 'stimulus'])['lick']
for d in range(1, 6, 1):
    s = 'last 20% SAT ' + str(d)    
    sat = firstlast20_lickfreq[c &(firstlast20_lickfreq['daytime'] == s)].set_index(['condition', 'animal', 'stimulus'])['lick']
    tmpdiffs.append((sat - acc).rename('SAT ' + str(d)))
tmpdiffs = pd.concat(tmpdiffs, axis=1)
tmpdiffs = tmpdiffs.stack().reset_index()
tmpdiffs = tmpdiffs.rename({'level_3':'day', 0:'lick'}, axis=1)

In [ ]:
g = sns.catplot(tmpdiffs, y='lick', x='condition',kind='bar', hue='day', col='stimulus', errorbar='se',err_kws={'lw':1, 'zorder':0}, alpha=0.3, palette='rocket',
                col_order=['blank', 'stimulus'], order=CO)

for name, ax in g.axes_dict.items():
    g.axes_dict[name].set_title(name)
    g.axes_dict[name].set_xticklabels(g.axes_dict[name].get_xticklabels(), rotation=45, ha='right')
    sns.stripplot(tmpdiffs[tmpdiffs['stimulus'] == name], ax=ax,x="condition", y="lick", hue='day',size=5, dodge=True, legend=False, jitter=False,palette='rocket',
                 order=CO)
blsegs = []
stsegs = []
for i in range(len(CO)):
    cdfilt = tmpdiffs[tmpdiffs['condition'] ==  CO[i]]
    for a in cdfilt["animal"].unique():
        anfilt = cdfilt[cdfilt["animal"] == a]
        
        for stim in ['stimulus', 'blank']:
            pts = []
            for j in range(1,6,1):        
                c = (anfilt["day"] == 'SAT ' + str(j)) & (anfilt['stimulus'] == stim)
                op = list(anfilt[c]["lick"])
                if (len(op) != 0):
                    pts.append([i + 1/(6)*(j-(6)/2), op[0]])
            if(len(pts) != 0):
                if stim =='stimulus':
                    stsegs.append(pts)
                else:
                    blsegs.append(pts)

bllinesegs = LineCollection(blsegs,color='k', lw=0.5, zorder=1)
g.axes_dict['blank'].add_collection(bllinesegs)
stlinesegs = LineCollection(stsegs,color='k', lw=0.5, zorder=1)
g.axes_dict['stimulus'].add_collection(stlinesegs)

g.axes[0][0].set_ylabel('Lick freq diff\n(SAT end - ACC end)')
g.axes[0][0].set_ylim([-10,10])

In [ ]:
learned = list(firstlast20_perf[(firstlast20_perf['daytime'] == o[0]) & (firstlast20_perf['lick'] > 0)]['animal'])#.groupby('condition').count()['lick'].to_clipboard()
firstlast20_perf[(firstlast20_perf.apply(lambda x: x['animal'] in learned, axis=1)) & (firstlast20_perf['daytime'] == o[3])& (firstlast20_perf['lick'] > 0)].groupby('condition').count()['lick'].to_clipboard()

In [ ]:
tmp = (firstlast20_perf[(firstlast20_perf['daytime'] == o[1])].set_index(['condition', 'animal'])['lick'] -  firstlast20_perf[(firstlast20_perf['daytime'] == o[0])].set_index(['condition', 'animal'])['lick']).reset_index()
g = sns.catplot(tmp, y='lick', x='condition',kind='bar', hue='condition', errorbar='se',err_kws={'lw':1, 'zorder':0}, alpha=0.3, order=CO)
sns.swarmplot(tmp, x="condition", y="lick", hue='condition',size=5, dodge=True, legend=False, order=CO)
g.axes[0][0].set_ylabel('Performance diff\n(test start - train end)')
g.axes[0][0].set_xticklabels(g.axes[0][0].get_xticklabels(), rotation=45)
g.axes[0][0].set_ylim([-5,5])

In [ ]:
tmp = (firstlast20_perf[(firstlast20_perf['daytime'] == o[1])].set_index(['condition', 'animal'])['lick'] -  firstlast20_perf[(firstlast20_perf['daytime'] == o[0])].set_index(['condition', 'animal'])['lick']).reset_index()
tmp = tmp.dropna()
stats.mannwhitneyu(tmp[tmp['condition'] == 'control']['lick'], tmp[tmp['condition'] == 'contra lesion']['lick'])

In [ ]:
# ant_means.to_csv(rf'D:\analysis\ant_means{r[0]}-{r[1]}_{time_bin}.csv', index=False)
# ant_perf.to_csv(rf'D:\analysis\ant_perf{r[0]}-{r[1]}_{time_bin}.csv', index=False)
# counts.to_csv(rf'D:\analysis\counts{r[0]}-{r[1]}_{time_bin}.csv', index=False)
# daytots.to_csv(rf'D:\analysis\daytots{r[0]}-{r[1]}_{time_bin}.csv', index=False)
# hourtots.to_csv(rf'D:\analysis\hourtots{r[0]}-{r[1]}_{time_bin}.csv', index=False)

# antfiltmeans.to_csv(rf'D:\analysis\antfiltmeans{r[0]}-{r[1]}_{time_bin}_{mins["min_trials"]}_{mins["min_blank"]}_{mins["min_stimulus"]}.csv', index=False)
# antfiltperf.to_csv(rf'D:\analysis\antfiltperf{r[0]}-{r[1]}_{time_bin}_{mins["min_trials"]}_{mins["min_blank"]}_{mins["min_stimulus"]}.csv', index=False)
# filtcounts.to_csv(rf'D:\analysis\filtcounts{r[0]}-{r[1]}_{time_bin}_{mins["min_trials"]}_{mins["min_blank"]}_{mins["min_stimulus"]}.csv', index=False)
# daytotsfilt.to_csv(rf'D:\analysis\daytotsfilt{r[0]}-{r[1]}_{time_bin}_{mins["min_trials"]}_{mins["min_blank"]}_{mins["min_stimulus"]}.csv', index=False)
# hourtotsfilt.to_csv(rf'D:\analysis\hourtotsfilt{r[0]}-{r[1]}_{time_bin}_{mins["min_trials"]}_{mins["min_blank"]}_{mins["min_stimulus"]}.csv', index=False)

# nthpart_lickfreq.to_csv(rf'D:\analysis\antfiltperf{r[0]}-{r[1]}_{time_bin}_{mins["min_trials"]}_{mins["min_blank"]}_{mins["min_stimulus"]}.csv', index=False)
# nthpart_counts.to_csv(rf'D:\analysis\antfiltperf{r[0]}-{r[1]}_{time_bin}_{mins["min_trials"]}_{mins["min_blank"]}_{mins["min_stimulus"]}.csv', index=False)
# nthpart_perf.to_csv(rf'D:\analysis\antfiltperf{r[0]}-{r[1]}_{time_bin}_{mins["min_trials"]}_{mins["min_blank"]}_{mins["min_stimulus"]}.csv', index=False)


In [ ]:
antfiltmeans['Time (hr)'] = antfiltmeans['Time (hr)'] + 2
antfiltperf['Time (hr)'] = antfiltperf['Time (hr)'] + 2
hourtotsfilt['Time (hr)'] = hourtotsfilt['Time (hr)'] + 2

antfiltmeans = antfiltmeans[antfiltmeans['Time (hr)'] < 48]
antfiltperf = antfiltperf[antfiltperf['Time (hr)'] < 48]
hourtotsfilt = hourtotsfilt[hourtotsfilt['Time (hr)'] < 48]


In [ ]:
antfiltmeans.loc[antfiltmeans['condition'].str.contains('retrain'), 'Time (hr)'] += 48 + 24
antfiltmeans.loc[antfiltmeans['condition'].str.contains('test'), 'Time (hr)'] += 48 + 48 + 24 + 24
antfiltmeans.loc[(antfiltmeans['condition'].str.contains('sham')) |
                 (antfiltmeans['condition'].str.contains('ipsi lesion')), 'condition'] = 'sham and ipsi lesion' + antfiltmeans.loc[(antfiltmeans['condition'].str.contains('sham')) |
                 (antfiltmeans['condition'].str.contains('ipsi lesion')), 'condition'].str.lstrip('shamip').str.lstrip(' ').str.lstrip('lesion')

antfiltperf.loc[antfiltperf['condition'].str.contains('retrain'), 'Time (hr)'] += 48 + 24
antfiltperf.loc[antfiltperf['condition'].str.contains('test'), 'Time (hr)'] += 48 + 48 + 24 + 24
antfiltperf.loc[(antfiltperf['condition'].str.contains('sham')) |
                 (antfiltperf['condition'].str.contains('ipsi lesion')), 'condition'] = 'sham and ipsi lesion' + antfiltperf.loc[(antfiltperf['condition'].str.contains('sham')) |
                 (antfiltperf['condition'].str.contains('ipsi lesion')), 'condition'].str.lstrip('shamip').str.lstrip(' ').str.lstrip('lesion')

hourtotsfilt.loc[hourtotsfilt['condition'].str.contains('retrain'), 'Time (hr)'] += 48 + 24
hourtotsfilt.loc[hourtotsfilt['condition'].str.contains('test'), 'Time (hr)'] += 48 + 48 + 24 + 24
hourtotsfilt.loc[(hourtotsfilt['condition'].str.contains('sham')) |
                 (hourtotsfilt['condition'].str.contains('ipsi lesion')), 'condition'] = 'sham and ipsi lesion' + hourtotsfilt.loc[(hourtotsfilt['condition'].str.contains('sham')) |
                 (hourtotsfilt['condition'].str.contains('ipsi lesion')), 'condition'].str.lstrip('shamip').str.lstrip(' ').str.lstrip('lesion')

In [ ]:
ylim = [-0.1, 10]
xlim = [-24, 192]
xticklab = ['', -24, -12, 0, 12, 24, 36, 48, '', 0, 12, 24, 36, 48, '', 0, 12, 24, 36, 48]
c = 'contra lesion'
g = plot_ant_lickfreq(antfiltmeans[(antfiltmeans['condition'].str.contains(c + ' train')) & ~(antfiltmeans['animal'].str.contains('PTE1'))]
                      ,y='lick',aspect=2,lw=1, ms=7,col='condition',
        title='Contra lesion',legend=False, xlim=xlim, ylim=ylim, errorbar='se', err_style='bars', err_kws={"lw":0.5},
        wspace=0.15,style=False)
sns.lineplot(antfiltmeans[(antfiltmeans['condition'].str.contains(c + ' retrain')) & ~(antfiltmeans['animal'].str.contains('PTE1'))]
                      ,y='lick',lw=1, ms=7,ax=g.axes[0][0],x='Time (hr)',
        errorbar='se', err_style='bars', err_kws={"lw":0.5},hue='stimulus', 
        hue_order=['blank', 'stimulus'], palette=['red', 'green'], mec=None, marker='o', 
        legend=False)
sns.lineplot(antfiltmeans[(antfiltmeans['condition'].str.contains(c + ' test')) & ~(antfiltmeans['animal'].str.contains('PTE1'))]
                      ,y='lick',lw=1, ms=7,ax=g.axes[0][0],x='Time (hr)',
        errorbar='se', err_style='bars', err_kws={"lw":0.5},hue='stimulus', 
        hue_order=['blank', 'stimulus'], palette=['red', 'green'], mec=None, marker='o', 
        legend=False)
g.axes[0][0].set_xticklabels(xticklab)
xticks = g.axes[0][0].xaxis.get_major_ticks()
xticks[8].set_visible(False)
xticks[14].set_visible(False)
xticks = g.axes[0][0].xaxis.get_minor_ticks()
for i in [13, 14, 15, 16, 25, 26, 27, 28]:
        xticks[i].set_visible(False)
g.axes[0][0].text(12, -2, 'Train', ha='center')
g.axes[0][0].text(60, -0.6, '//', ha='center', va='bottom', fontsize=24)
g.axes[0][0].text(144-12, -0.6, '//', ha='center', va='bottom', fontsize=24)
g.axes[0][0].text(96, -2, '5d test', ha='center')
g.axes[0][0].text(168, -2, '21d test', ha='center')
g.axes[0][0].set_xlabel(g.axes[0][0].get_xlabel(), labelpad=28)

g = plot_ant_perf(antfiltperf[(antfiltperf['condition'].str.contains(c + ' train')) & ~(antfiltperf['animal'].str.contains('PTE1'))]
                  , ylabel="Performance\n$L_s - L_b$", aspect=2,lw=1, ms=7,
                title='Contra lesion',xlim=xlim,col='condition',
                legend=True, ylim=[-8, 8], errorbar='se', err_style='bars', err_kws={"lw":0.5},
                wspace=0.25
                )

sns.lineplot(antfiltperf[(antfiltperf['condition'].str.contains(c + ' retrain')) & ~(antfiltperf['animal'].str.contains('PTE1'))]
                      ,y='lick',lw=1, ms=7,ax=g.axes[0][0],x='Time (hr)',
        errorbar='se', err_style='bars', err_kws={"lw":0.5}, mec=None, marker='o', color='k',
        legend=False)
sns.lineplot(antfiltperf[(antfiltperf['condition'].str.contains(c + ' test')) & ~(antfiltperf['animal'].str.contains('PTE1'))]
                      ,y='lick',lw=1, ms=7,ax=g.axes[0][0],x='Time (hr)',
        errorbar='se', err_style='bars', err_kws={"lw":0.5},color='k', mec=None, marker='o', 
        legend=False)
g.axes[0][0].set_xticklabels(xticklab)
xticks = g.axes[0][0].xaxis.get_major_ticks()
xticks[8].set_visible(False)
xticks[14].set_visible(False)
xticks = g.axes[0][0].xaxis.get_minor_ticks()
for i in [13, 14, 15, 16, 25, 26, 27, 28]:
        xticks[i].set_visible(False)
g.axes[0][0].text(12, -12, 'Train', ha='center')
g.axes[0][0].text(60, -10.6, '//', ha='center', va='bottom', fontsize=24)
g.axes[0][0].text(144-11, -10.6, '//', ha='center', va='bottom', fontsize=24)
g.axes[0][0].text(96, -11, '5d test', ha='center')
g.axes[0][0].text(168, -11, '21d test', ha='center')
g.axes[0][0].set_xlabel(g.axes[0][0].get_xlabel(), labelpad=28)

g = plot_trial_hr(hourtotsfilt[(hourtotsfilt['condition'].str.contains(c  + ' train')) & ~(hourtotsfilt['animal'].str.contains('PTE1'))],
                  xlim=xlim, col='condition',fill=False,
               ylim=[0, 400], aspect=3, title='Contra lesion', ms=7, lw=1,err_kws={"lw":0.5}, ylabel='Number of trials',
               wspace=0.04)
sns.lineplot(hourtotsfilt[(hourtotsfilt['condition'].str.contains(c + ' retrain')) & ~(hourtotsfilt['animal'].str.contains('PTE1'))],
                      y='trial no',lw=1, ms=7,ax=g.axes[0][0],x='Time (hr)',
        errorbar='se', err_style='bars', err_kws={"lw":0.5},color='k', mec=None, marker='o', 
        legend=False)
sns.lineplot(hourtotsfilt[(hourtotsfilt['condition'].str.contains(c + ' test')) & ~(hourtotsfilt['animal'].str.contains('PTE1'))],
                      y='trial no',lw=1, ms=7,ax=g.axes[0][0],x='Time (hr)',
        errorbar='se', err_style='bars', err_kws={"lw":0.5},color='k', mec=None, marker='o', 
        legend=False)
g.axes[0][0].set_xticklabels(xticklab)
xticks = g.axes[0][0].xaxis.get_major_ticks()
xticks[8].set_visible(False)
xticks[14].set_visible(False)
xticks = g.axes[0][0].xaxis.get_minor_ticks()
for i in [13, 14, 15, 16, 25, 26, 27, 28]:
        xticks[i].set_visible(False)
g.axes[0][0].text(12, -85, 'Train', ha='center')
g.axes[0][0].text(60, -25, '//', ha='center', va='bottom', fontsize=24)
g.axes[0][0].text(144-12, -25, '//', ha='center', va='bottom', fontsize=24)
g.axes[0][0].text(96, -85, '5d test', ha='center')
g.axes[0][0].text(168, -85, '21d test', ha='center')
g.axes[0][0].set_xlabel(g.axes[0][0].get_xlabel(), labelpad=28)
for i in range(len(g.axes[0][0].lines)):
        if i % 2 == 0:
                g.axes[0][0].fill_between(g.axes[0][0].lines[i].get_data()[0], g.axes[0][0].lines[i].get_data()[1], 
                                        color='k', alpha=0.3)
# for name, ax in g.axes_dict.items():
#         ax.spines[['right']].set_visible(True)
#         ax.spines[['left']].set_visible(False)
#         ax.xaxis.set_visible(False)
#         ax.yaxis.tick_right()

In [ ]:
daytotsfilt = daytotsfilt[daytotsfilt['Day'] < 2]
minday = int(daytotsfilt["Day"].min())
daytotsfilt["Day"] = daytotsfilt["Day"].apply(lambda x: day_to_label(minday, x))

daytotsfilt['time'] = daytotsfilt['condition'].str.lstrip('shamcontripl').str.lstrip(' ').str.lstrip('leison').str.lstrip(' ')
daytotsfilt.loc[(daytotsfilt['condition'].str.contains('sham')) |
                 (daytotsfilt['condition'].str.contains('ipsi lesion')), 'condition'] = 'sham and ipsi lesion' + daytotsfilt.loc[(daytotsfilt['condition'].str.contains('sham')) |
                 (daytotsfilt['condition'].str.contains('ipsi lesion')), 'condition'].str.lstrip('shamip').str.lstrip(' ').str.lstrip('lesion')
daytotsfilt['cond'] = daytotsfilt['condition'].str.rstrip('trainstre').str.rstrip(' ')


In [ ]:
ordr = ['control', 'sham and ipsi lesion', 'contra lesion']

g = plot_trial_bar(daytotsfilt[daytotsfilt['time'] =='train'], hue='Day',x='cond',col=None,
                   order=ordr, hue_order=['ACC 0', 'SAT 1', 'SAT 2'],aspect=1,
                   color=None, dodge='auto', legend=False,fill=True, linewidth=1,
                   palette=['#DBDBDB', '#C8CFFF', '#FCAEF3'],
                   width=.8, ylim=[0, 1300], ymajmult=200)
g.axes[0][0].set_xticklabels(['No lesion', 'Sham lesion', 'Contra leison'], rotation=45, ha='right')
g.axes[0][0].set_title('Train')

g = plot_trial_bar(daytotsfilt[daytotsfilt['time'] =='retrain'], hue='Day',x='cond',col=None,
                   order=ordr, hue_order=['SAT 1', 'SAT 2'],aspect=1,
                   color=None, dodge='auto', legend=False,fill=True, linewidth=1,
                   palette=['#C8CFFF', '#FCAEF3'],
                   width=.8, ylim=[0, 1300], ymajmult=200)
g.axes[0][0].set_xticklabels(['No lesion', 'Sham lesion', 'Contra leison'], rotation=45, ha='right')
g.axes[0][0].set_title('5d test')

g = plot_trial_bar(daytotsfilt[daytotsfilt['time'] =='test'], hue='Day',x='cond',col=None,
                   order=ordr, hue_order=['SAT 1', 'SAT 2'],aspect=1,
                   color=None, dodge='auto', legend=False,fill=True, linewidth=1,
                   palette=['#C8CFFF', '#FCAEF3'],
                   width=.8, ylim=[0, 1300], ymajmult=200)
g.axes[0][0].set_xticklabels(['No lesion', 'Sham lesion', 'Contra leison'], rotation=45, ha='right')
g.axes[0][0].set_title('21d test')

In [ ]:
res = []
res1 = []
daytotsfilt = daytotsfilt.sort_values(['condition', 'animal', 'Day'])

filt = ~(daytotsfilt['condition'].str.contains('-')) & ~(daytotsfilt['condition'].str.contains('SAT'))
ind = daytotsfilt[filt][["Day", "time"]].apply(tuple, axis=1).unique()
#ind = daytotsfilt[filt]['time'].unique()
conds = daytotsfilt[filt]['cond'].unique()

for d, t in ind:
    test = []
    tcond = daytotsfilt['time'] == t
    dcond = daytotsfilt['Day'] == d
    for c in conds:
        ccond = daytotsfilt["cond"] == c
        dat = daytotsfilt[ccond & tcond & dcond]['trial no']
        if len(dat) > 1:
            test.append(dat)
    #print(t, test)
    if len(test) > 0:
        res.append(stats.f_oneway(*test))
        res1.append(stats.tukey_hsd(*test).pvalue)
    else:
        res.append(pd.NA)
        res1.append(pd.NA)

pd.DataFrame(res, columns=["statistic", "pvalue"], index=pd.Series(ind).apply(lambda x: x[1] + ", " + x[0]))


In [ ]:
pd.DataFrame(res1, columns=["pvalue"], index=pd.Series(ind).apply(lambda x: x[1] + ", " + x[0]))

In [ ]:
xlim=[0, 48]
for c in antfiltmeans['cond'].unique():
    #if 'lesion train' in c:
        tmp = antfiltmeans[antfiltmeans['cond'] == c]
        for a in tmp["animal"].unique():               
             if 'PB716' in a:
                g = plot_ant_lickfreq(antfiltmeans[(antfiltmeans["animal"] == a)],y='lick',aspect=2,lw=1, ms=7, title=a,
                        legend=False, xlim=xlim, ylim=[-0.1, 10], errorbar='se', err_style='bars', err_kws={"lw":0.5})
                g.axes[0][0].add_patch(Rectangle((48, -0.1), 120, 10 - (-0.1), color="#D0D0D0", alpha=0.5, 
                                zorder=0, fill=True))
                g = plot_ant_perf(antfiltperf[(antfiltperf["animal"] == a)], ylabel="Performance", aspect=2,lw=1, ms=7,
                                xlim=xlim, legend=False, ylim=[-10, 10], title='',errorbar='se', err_style='bars', err_kws={"lw":0.5})
                g.axes[0][0].add_patch(Rectangle((48, -10), 120, 20, color="#D0D0D0", alpha=0.5, 
                                zorder=0, fill=True))
                g = plot_trial_hr(hourtotsfilt[(hourtotsfilt["animal"] == a)], xlim=xlim, 
                                ylim=[0, 800], aspect=6, title='', ms=7, lw=1,err_kws={"lw":0.5}, ylabel='')
                g.axes[0][0].add_patch(Rectangle((48, 0), 120, 800, color="#D0D0D0", alpha=0.5, 
                                zorder=0, fill=True))
#g.figure.suptitle(c, y=1.05, x=0.5, fontsize=25)

In [ ]:
minday = int(nthpart_lickfreq["Day"].min())
nthpart_lickfreq["Day"] = nthpart_lickfreq["Day"].apply(lambda x: day_to_label(minday, x))

In [ ]:
for day in nthpart_lickfreq["Day"].unique():
    if day in ['SAT 1', 'SAT 2']:
        x=[1, 2]
        i=0
        conds = nthpart_lickfreq["condition"].unique()
        conds.sort()
        dayfilt = nthpart_lickfreq[nthpart_lickfreq["Day"] == day]
        for c in conds:
            if 'lesion retrain' in c:
                condfilt = dayfilt[(dayfilt["condition"]==c) & ~(dayfilt['animal'].str.contains('PTE1'))]
                if not condfilt.empty:
                    g = sns.catplot(condfilt, y="lick",x="stimulus", order=["stimulus", "blank"],kind="strip", 
                            hue="stimulus", palette=["green", "red"], 
                            hue_order=["stimulus", "blank"],
                            col="condition", jitter=False, aspect=0.5, legend=False)
                    segs = []
                    for a in condfilt["animal"].unique():
                            anfilt = condfilt[condfilt["animal"] == a]
                            bl = anfilt[anfilt["stimulus"] == "blank"]["lick"].reset_index(drop=True)
                            st = anfilt[anfilt["stimulus"] == "stimulus"]["lick"].reset_index(drop=True)
                            #print(a, day, 'blank', bl, '\nstimulus', st)
                            if (len(bl) != 0) & (len(st) != 0):
                                add=[[0, st[0]], [1,bl[0]]]
                            segs.append(add)
                    linesegs = LineCollection(segs,color='k', lw=1)
                    g.axes_dict[c].add_collection(linesegs)
                    g.axes_dict[c].set_ylim([-2, 12])
                    g.axes_dict[c].set_title(f"{day} {c}")
                    g.axes_dict[c].tick_params(axis='x', pad=-3)
                    g.axes_dict[c].set_xticklabels(g.axes_dict[c].get_xticklabels(), rotation=45, ha="right")
                    i+=1
        #g.figure.suptitle(day, y=1.05)

In [ ]:
res = []
nthpart_lickfreq = nthpart_lickfreq.sort_values(['condition', 'animal', 'day_delta'])

stimcond = nthpart_lickfreq["stimulus"] == "stimulus"
blankcond = nthpart_lickfreq["stimulus"] == "blank"

conds = nthpart_lickfreq["condition"].unique()
#filt = (nthpart_lickfreq['condition'].str.contains('control'))
#filt = filt & ~((nthpart_lickfreq['animal'].str.contains('PNL6'))| (nthpart_lickfreq['animal'].str.contains('PNL7')))
filt =  ((nthpart_lickfreq["Day"] == 'ACC 0') | (nthpart_lickfreq["Day"] == 'SAT 2') | (nthpart_lickfreq["Day"] == 'SAT 1'))
ind = nthpart_lickfreq[filt][["Day", "condition"]].apply(tuple, axis=1).unique()

for day, c in ind:
    daycond = nthpart_lickfreq["Day"] == day
    ccond = nthpart_lickfreq["condition"] == c
    stim = nthpart_lickfreq[filt & stimcond & daycond & ccond]["lick"]
    san = nthpart_lickfreq[filt & stimcond & daycond & ccond][['animal',"lick"]]
    blank = nthpart_lickfreq[filt & blankcond & daycond & ccond]["lick"]
    ban = nthpart_lickfreq[filt & blankcond & daycond & ccond][['animal',"lick"]]
    #print(day, c,'stim', san,'blank', ban, '\n')
    res.append(stats.wilcoxon(stim, blank))

pd.DataFrame(res, columns=["statistic", "pvalue"], index=pd.Series(ind).apply(lambda x: x[1] + ", " + x[0]))

In [ ]:
minday = int(nth_part_data["Day"].min())
nth_part_data["Day"] = nth_part_data["Day"].apply(lambda x: day_to_label(minday, x))

In [ ]:
res = []
nth_part_data = nth_part_data.sort_values(['condition', 'animal', 'day_delta'])

stimcond = nth_part_data["stimulus"] == "stimulus"
blankcond = nth_part_data["stimulus"] == "blank"

conds = nth_part_data["animal"].unique()
filt = (nth_part_data['condition'].str.contains('contra lesion'))
#filt = filt & ~((nthpart_lickfreq['animal'].str.contains('PNL6'))| (nthpart_lickfreq['animal'].str.contains('PNL7')))
filt = filt & ((nth_part_data["Day"] == 'ACC 0') | (nth_part_data["Day"] == 'SAT 2') | (nth_part_data["Day"] == 'SAT 1'))
ind = nth_part_data[filt][["Day", "animal"]].apply(tuple, axis=1).unique()

for day, c in ind:
    daycond = nth_part_data["Day"] == day
    ccond = nth_part_data["animal"] == c
    stim = nth_part_data[filt & stimcond & daycond & ccond]["lick"]
    #san = nthpart_lickfreq[filt & stimcond & daycond & ccond][['animal',"lick"]]
    blank = nth_part_data[filt & blankcond & daycond & ccond]["lick"]
    #ban = nthpart_lickfreq[filt & blankcond & daycond & ccond][['animal',"lick"]]
    print(day, c, len(stim), len(blank))
    if len(stim) > 0 and len(blank) > 0:
        res.append(stats.mannwhitneyu(stim, blank))
    else:
        res.append([pd.NA, pd.NA])

ind = (nth_part_data[filt]['Day'] + '  ' + nth_part_data[filt]["animal"].str.replace(' ', '  ')).str.split('  ').apply(tuple).unique()
sig = pd.DataFrame(res, columns=["statistic", "pvalue"], index=pd.MultiIndex.from_tuples(ind,names=['Day', 'animal', 'cond']))
sig['pvalue'].unstack(level='animal').sort_index()

In [ ]:
ind = (nth_part_data[filt]['Day'] + '  ' + nth_part_data[filt]["animal"].str.replace(' ', '  ')).str.split('  ').apply(tuple)
pd.MultiIndex.from_tuples(ind)
#tmp = pd.Series(ind).apply(lambda x: x[1] + "  " + x[0]).str.split('  ').to_numpy()
#pd.MultiIndex.from_arrays(tmp)

#### Slopes


In [ ]:

# lick freq slope across trial - y2 - y1 / (x2- x1)
# (x2-x1 will always == 100ms because of sampling)
# so this should be lickfreq.diff()/100
def getSlopes(data, kp, filt, r, samplerate):
    slopes = data.copy()
    kpdf = slopes.groupby(['condition', 'animal','stimulus','Time (hr)'])[kp].first()
    if r != None:
        rangefilt = (data["Time (ms)"] > r[0]) & (data['Time (ms)']  <= r[1])
        slopes = data[rangefilt]
    if not filt.empty:
        slopes = slopes[filt]
    slopes = slopes.set_index(['condition', 'animal', 'Time (hr)', 'Time (ms)', 'stimulus'])
    # get slope for each animal for each time bin
    # (diff = y2 - y1, samplerate (should be constant) = x2 - x1)
    slopes = slopes.groupby(['condition', 'animal', 'Time (hr)', 'stimulus'])['lick'].diff()/samplerate
    slopes = slopes.reset_index()
    # mean slope across time range
    slopes = slopes.groupby(['condition', 'animal','stimulus','Time (hr)']).mean()
    return pd.concat([slopes, kpdf], axis=1).reset_index()


In [ ]:
# separate consummatory and anticipatory slope - whole trial slope doesn't show difference because animals are doing something different in anticipatory and consummatory periods
# and whole trial slope collapses that information
# cond = (filtmeans['condition'].str.contains('lesion'))
kp = []
# slopes = getSlopes(filtmeans, cond, None, 100)

slopesant = getSlopes(filtmeans,kp, pd.Series(), (100, 1000), 100)
slopesant['slope type'] = 'anticipatory'
slopesant['Time (hr)'] = slopesant['Time (hr)'] + 2

slopescon = getSlopes(filtmeans, kp, pd.Series(), (1000, 1700), 100)
slopescon['slope type'] = 'consummatory'
slopescon['Time (hr)'] = slopescon['Time (hr)'] + 2

slopespre = getSlopes(filtmeans, kp, pd.Series(), (-300, 0), 100)
slopespre['slope type'] = 'pre-puff'
slopespre['Time (hr)'] = slopespre['Time (hr)'] + 2


In [ ]:

#slopesnext = getSlopes(filtmeans, cond, (1800, 2000), 100)
#slopesnext['slope type'] = 'next trial anticipatory'
# g = plot_ant_lickfreq(slopes, ylim=[-0.0025, 0.0025], xlim=[-24, 28], errorbar='se', err_style='bars', err_kws={'lw':0.5})
# for (n,ax) in g.axes_dict.items():
#     ax.axhline(y=0, xmin=0, xmax=1, ls="-", color="black", zorder=10, lw=1)
# g.fig.suptitle('whole trial')
hord = ['Ipsi-lesion', 'Contra-lesion', 'retrain - train']
pal =  sns.color_palette('husl', n_colors=3)#['blue', 'teal',          'powderblue']
hue = 'condition'
col = 'stimulus'
cord = ['stimulus', 'blank']
xlim=[-24,48]

g = plot_ant_lickfreq(slopesant, ylim=[-0.008, 0.008], xlim=xlim, col=col,col_order=cord,hue=hue,hue_order=hord,palette=pal,
                      errorbar='se', err_style='bars', err_kws={'lw':0.5},ymajmult=0.004, yminmult=0.001,
                      )
for (n,ax) in g.axes_dict.items():
     ax.axhline(y=0, xmin=0, xmax=1, ls="-", color="black", zorder=10, lw=1)
g.fig.suptitle('anticipatory (100-1000ms from puff onset)', fontsize=20, y=1.01)

g = plot_ant_lickfreq(slopescon, ylim=[-0.008, 0.008], xlim=xlim,col=col,col_order=cord,hue=hue,hue_order=hord,palette=pal,
                      errorbar='se', err_style='bars', err_kws={'lw':0.5},ymajmult=0.004, yminmult=0.001,
                      )
for (n,ax) in g.axes_dict.items():
     ax.axhline(y=0, xmin=0, xmax=1, ls="-", color="black", zorder=10, lw=1)
g.fig.suptitle('consummatory (1000-1700 ms from puff onset)', fontsize=20, y=1.01)

# g = plot_ant_lickfreq(slopespre, ylim=[-0.005, 0.005], xlim=[-24, 120], errorbar='se',err_style='bars', err_kws={'lw':0.5},ymajmult=0.004, yminmult=0.001)
# for (n,ax) in g.axes_dict.items():
#      ax.axhline(y=0, xmin=0, xmax=1, ls="-", color="black", zorder=10, lw=1)
# g.fig.suptitle('pre-puff')

#g = plot_ant_lickfreq(slopesnext, ylim=[-0.005, 0.05], xlim=[-24, 28], errorbar='se',err_style='bars', err_kws={'lw':0.5},col_wrap=4)
#for (n,ax) in g.axes_dict.items():
#     ax.axhline(y=0, xmin=0, xmax=1, ls="-", color="black", zorder=10, lw=1)
#g.fig.suptitle('next trial anticipatory')



#### Correlations


compare {total trials, stim trials, blank trials} - {perf, stim, blank, iti, iti learn time}
        iti - {perf, stim, blank}


In [ ]:
#minday = pokestarts["Day"].min()
#diffs_by_time["Day"] = diffs_by_time["Day"].apply(lambda x: day_to_label(minday, x))
kp = ['Sex', 'Cage', 'ITI', 'lick', 'learn time auto', 'acc time']
last20iti = get_nth_part_day(diffs_by_time, 5, 1)
counts = trial_counts(last20iti, ['condition', 'animal', 'Day', 'diff num'], kp,'trial no')
counts_under10 = trial_counts(last20iti[last20iti['ITI'] < 10000],  ['condition', 'animal', 'Day', 'diff num'],kp,'trial no')
counts_close  = trial_counts(last20iti[last20iti['ITI'] < 3500], ['condition', 'animal', 'Day', 'diff num'], kp,'trial no')

counts = counts.sort_values(['condition', 'Day', 'diff num', 'learn time auto'])
counts_under10 = counts_under10.sort_values(['condition', 'Day', 'diff num', 'learn time auto'])
counts_close = counts_close.sort_values(['condition', 'Day', 'diff num', 'learn time auto'])
fracclose = ((counts_close.set_index(['condition', 'animal', 'Day', 'diff num', 'learn time auto'])['trial no']) / (counts_under10.set_index(['condition', 'animal', 'Day', 'diff num', 'learn time auto'])['trial no']))
fracclose = fracclose.reset_index()
fracclose = fracclose[fracclose['diff num'] == 1]
fracclose = fracclose.dropna()
corrs = nthpart_lickfreq.set_index(['condition', 'animal', 'Day'])
corrs['total trials'] = filtcounts.groupby(['condition', 'animal', "Day"])['trial no'].sum()
corrs['perf'] = nthpart_perf.set_index(['condition', 'animal', 'Day'])['lick']
corrs['ITI'] = diffs_by_time.groupby(['condition', 'animal', 'Day'])['ITI'].mean()
last20iti = get_nth_part_day(diffs_by_time, 5, 1)
corrs['ITIlast20'] = last20iti.groupby(['condition', 'animal', 'Day'])['ITI'].mean()
corrs['fracclose'] = fracclose.set_index(['condition','animal', 'Day'])['trial no']
corrs = corrs.reset_index().set_index(['condition', 'animal', 'Day', 'stimulus'])
corrs['trial no'] = filtcounts.groupby(['condition', 'animal', 'Day', 'stimulus'])['trial no'].sum()
corrs['ITI_stims'] = diffs_by_time.groupby(['condition', 'animal', 'Day', 'stimulus'])['ITI'].mean()

corrs = corrs.reset_index().set_index(['condition', 'animal'])
corrs['learn time'] = diffs_by_time.groupby(['condition', 'animal'])['learn time auto'].first()

corrs = corrs.reset_index()


Perf at Last 20% vs Perf at Last 20% retrain

In [ ]:
def lin(x, b, a):
    return a*x + b

def pw(x, a, b):
    return np.exp(a)*(x**b)

f, ax = plt.subplots(1,1, figsize=(5,5))
            
xdata = firstlast20_perf[(firstlast20_perf['condition']=='control') & (firstlast20_perf['daytime'] == 'train last 20% SAT 2') & ~((firstlast20_perf['animal'] == 'PTE1') | (firstlast20_perf['animal']== 'PTE2')| (firstlast20_perf['animal']== 'PB726'))]['lick']
ydata = firstlast20_perf[(firstlast20_perf['condition']=='control') & (firstlast20_perf['daytime'] == 'retrain first 20% SAT 1')]['lick']
res = stats.linregress(xdata, y=ydata)
popt = (res.intercept, res.slope)
print(xdata)
ax.plot(xdata, ydata, 'ko')
ax.plot(xdata, lin(xdata, *popt), 'r-', lw=1, label='y = %5.3fx + %5.3f' % (res.slope, res.intercept))
ax.set_xlim([-2, 6.5])
ax.set_ylim([-2, 6.5])
ax.set_xlabel('Last 20% train performance')
ax.set_ylabel('First 20% retrain performance')
ax.spines['right'].set_visible(False)
ax.xaxis.set_major_locator(ticker.MultipleLocator(2))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(0.5))
ax.yaxis.set_major_locator(ticker.MultipleLocator(2))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(0.5))
ax.set_title('control')
#f.legend(loc='lower right', )
print('%s %s -  slope: %5.3f, y-int: %5.3f,R:%1.5f, pval:%1.3f' % (c,day,res.slope, res.intercept,res.rvalue, res.pvalue))
#ax.set_title(day)  
#f.suptitle(c)

# corrs_tot = corrs[(corrs['stimulus'] == 'stimulus') & (corrs['Day'] == "SAT 1")].sort_values('total trials')
# xdata = corrs_tot['total trials']
# ydata = corrs_tot['perf']
# res = stats.linregress(xdata, y=ydata)
# popt = (res.intercept, res.slope)
# f, axs = plt.subplots(1,1, figsize=(5,5))
# axs.plot(corrs_tot['total trials'], corrs_tot['perf'], 'ko', label='data')
# axs.plot(corrs_tot['total trials'],lin(corrs_tot['total trials'], *popt), 'r-',lw=1)
# print(res)


Trial no vs ITI (strong correlation, inverse (1/x) relationship)

due to mathematical features of ITI calculation


In [ ]:
def lin(x, b, a):
    return a*x + b

def pw(x, a, b):
    return np.exp(a)*(x**b)

conds = corrs['cond'].unique()
for c in conds:
    if 'lesion' in c:
        corrs_tot = corrs[(corrs['stimulus'] == 'stimulus') & (corrs['cond'] == c)]
        if len(corrs_tot['animal'].unique()) > 2:
            days = corrs_tot['Day'].unique()
            f, ax = plt.subplots(2,len(days), figsize=(5*(len(days)),10))
            i=0
            for day in days:
                plt_data = corrs_tot[(corrs_tot['Day'] == day)].sort_values('total trials')        
                if len(plt_data['animal'].unique()) > 2:
                    xdata = np.log(plt_data['total trials'])
                    ydata = np.log(plt_data['ITIlast20'])
                    drops = xdata.isna() | ydata.isna()
                    xdata = xdata[~drops]
                    ydata = ydata[~drops]
                    res = stats.linregress(xdata, y=ydata)
                    popt = (res.intercept, res.slope)
                    ax[0][i].plot(np.log(plt_data['total trials']), np.log(plt_data['ITIlast20']), 'ko',label='data')
                    ax[0][i].plot(xdata, lin(xdata, *popt), 'r-',lw=1,label='fit: a=%5.3f, b=%5.3f' % tuple(popt))
                    ax[1][i].plot(plt_data['total trials'], plt_data['ITIlast20'], 'ko')
                    ax[1][i].plot(plt_data['total trials'], pw(plt_data['total trials'], *popt), 'r-', lw=1)
                    print('%s %s -  slope: %5.3f, y-int: %5.3f,R:%1.5f, pval:%1.3E' % (c,day,res.slope, res.intercept,res.rvalue, res.pvalue))
                    ax[0][i].set_title(day)  
                    i+=1
            f.suptitle(c)
            f.tight_layout()
                    
    else:
        print(c)

corrs_tot = corrs[(corrs['stimulus'] == 'stimulus') & (corrs['Day'] == "ACC 0")].sort_values('total trials')
xdata = np.log(corrs_tot['total trials'])
ydata = np.log(corrs_tot['ITIlast20'])
drops = xdata.isna() | ydata.isna()
xdata = xdata[~drops]
ydata = ydata[~drops]
res = stats.linregress(xdata, y=ydata)
popt = (res.intercept, res.slope)
f, axs = plt.subplots(1,2, figsize=(10,5))
axs[0].plot(np.log(corrs_tot['total trials']), np.log(corrs_tot['ITIlast20']), 'ko', label='data')
axs[1].plot(corrs_tot['total trials'], corrs_tot['ITIlast20'], 'ko', label='data')
axs[0].plot(xdata, lin(xdata, *popt), 'r-', lw=1)
axs[1].plot(corrs_tot['total trials'],pw(corrs_tot['total trials'], *popt), 'c-',lw=1)
print(res)


Performance vs Fraction of trials with low ITI (no correlation)


In [ ]:
def lin(x, b, a):
    return a*x + b

def pw(x, a, b):
    return np.exp(a)*(x**b)



conds = corrs['cond'].unique()
for c in conds:
    if 'lesion' in c or c =='SAT':
        corrs_tot = corrs[(corrs['stimulus'] == 'blank') & (corrs['cond'] == c)]
        if len(corrs_tot['animal'].unique()) > 2:
            days = corrs_tot['Day'].unique()
            f, ax = plt.subplots(1,len(days), figsize=(5*(len(days)),5))
            i=0
            for day in days:
                plt_data = corrs_tot[(corrs_tot['Day'] == day)].sort_values('fracclose')        
                if len(plt_data['animal'].unique()) > 2:
                    xdata = plt_data['fracclose']
                    ydata = plt_data['lick']
                    res = stats.linregress(xdata, y=ydata)
                    popt = (res.intercept, res.slope)
                    ax[i].plot(plt_data['fracclose'], plt_data['lick'], 'ko')
                    ax[i].plot(plt_data['fracclose'], lin(plt_data['fracclose'], *popt), 'r-', lw=1)
                    print('%s %s -  slope: %5.3f, y-int: %5.3f,R:%1.5f, pval:%1.3E' % (c,day,res.slope, res.intercept,res.rvalue, res.pvalue))
                    ax[i].set_title(day)  
                    i+=1
            f.suptitle(c)
            f.tight_layout()
                    
    else:
        print(c)

corrs_tot = corrs[(corrs['stimulus'] == 'stimulus') & (corrs['Day'] == "SAT 1")].sort_values('fracclose')
xdata = corrs_tot['fracclose']
ydata = corrs_tot['lick']
res = stats.linregress(xdata, y=ydata)
popt = (res.intercept, res.slope)
f, axs = plt.subplots(1,1, figsize=(5,5))
axs.plot(corrs_tot['fracclose'], corrs_tot['perf'], 'ko', label='data')
axs.plot(corrs_tot['fracclose'],lin(corrs_tot['fracclose'], *popt), 'r-',lw=1)
print(res)


Trial no vs performance (no correlation)


In [ ]:
def lin(x, b, a):
    return a*x + b

def pw(x, a, b):
    return np.exp(a)*(x**b)



conds = corrs['cond'].unique()
for c in conds:
    if 'lesion' in c:
        corrs_tot = corrs[(corrs['stimulus'] == 'stimulus') & (corrs['cond'] == c)]
        if len(corrs_tot['animal'].unique()) > 2:
            days = corrs_tot['Day'].unique()
            f, ax = plt.subplots(1,len(days), figsize=(5*(len(days)),5))
            i=0
            for day in days:
                plt_data = corrs_tot[(corrs_tot['Day'] == day)].sort_values('total trials')        
                if len(plt_data['animal'].unique()) > 2:
                    xdata = plt_data['total trials']
                    ydata = plt_data['perf']
                    res = stats.linregress(xdata, y=ydata)
                    popt = (res.intercept, res.slope)
                    ax[i].plot(plt_data['total trials'], plt_data['perf'], 'ko')
                    ax[i].plot(plt_data['total trials'], lin(plt_data['total trials'], *popt), 'r-', lw=1)
                    print('%s %s -  slope: %5.3f, y-int: %5.3f,R:%1.5f, pval:%1.3E' % (c,day,res.slope, res.intercept,res.rvalue, res.pvalue))
                    ax[i].set_title(day)  
                    i+=1
            f.suptitle(c)
            f.tight_layout()
                    
    else:
        print(c)

corrs_tot = corrs[(corrs['stimulus'] == 'stimulus') & (corrs['Day'] == "SAT 1")].sort_values('total trials')
xdata = corrs_tot['total trials']
ydata = corrs_tot['perf']
res = stats.linregress(xdata, y=ydata)
popt = (res.intercept, res.slope)
f, axs = plt.subplots(1,1, figsize=(5,5))
axs.plot(corrs_tot['total trials'], corrs_tot['perf'], 'ko', label='data')
axs.plot(corrs_tot['total trials'],lin(corrs_tot['total trials'], *popt), 'r-',lw=1)
print(res)


Trial no vs lick to stim (no correlation)


In [ ]:
def lin(x, b, a):
    return a*x + b

def pw(x, a, b):
    return np.exp(a)*(x**b)



conds = corrs['cond'].unique()
for c in conds:
    if 'lesion' in c:
        corrs_tot = corrs[(corrs['stimulus'] == 'stimulus') & (corrs['cond'] == c)]
        if len(corrs_tot['animal'].unique()) > 2:
            days = corrs_tot['Day'].unique()
            f, ax = plt.subplots(1,len(days), figsize=(5*(len(days)),5))
            i=0
            for day in days:
                plt_data = corrs_tot[(corrs_tot['Day'] == day)].sort_values('total trials')        
                if len(plt_data['animal'].unique()) > 2:
                    xdata = plt_data['total trials']
                    ydata = plt_data['lick']
                    res = stats.linregress(xdata, y=ydata)
                    popt = (res.intercept, res.slope)
                    ax[i].plot(plt_data['total trials'], plt_data['lick'], 'ko')
                    ax[i].plot(plt_data['total trials'], lin(plt_data['total trials'], *popt), 'r-', lw=1)
                    print('%s %s -  slope: %5.3f, y-int: %5.3f,R:%1.5f, pval:%1.3E' % (c,day,res.slope, res.intercept,res.rvalue, res.pvalue))
                    ax[i].set_title(day)  
                    i+=1
            f.suptitle(c)
            f.tight_layout()
                    
    else:
        print(c)

corrs_tot = corrs[(corrs['stimulus'] == 'stimulus') & (corrs['Day'] == "SAT 1")].sort_values('total trials')
xdata = corrs_tot['total trials']
ydata = corrs_tot['lick']
res = stats.linregress(xdata, y=ydata)
popt = (res.intercept, res.slope)
f, axs = plt.subplots(1,1, figsize=(5,5))
axs.plot(corrs_tot['total trials'], corrs_tot['lick'], 'ko', label='data')
axs.plot(corrs_tot['total trials'],lin(corrs_tot['total trials'], *popt), 'r-',lw=1)
print(res)


Trial no vs lick to blank (no correlation)


In [ ]:
def lin(x, b, a):
    return a*x + b

def pw(x, a, b):
    return np.exp(a)*(x**b)



conds = corrs['cond'].unique()
for c in conds:
    if 'lesion' in c:
        corrs_tot = corrs[(corrs['stimulus'] == 'blank') & (corrs['cond'] == c)]
        if len(corrs_tot['animal'].unique()) > 2:
            days = corrs_tot['Day'].unique()
            f, ax = plt.subplots(1,len(days), figsize=(5*(len(days)),5))
            i=0
            for day in days:
                plt_data = corrs_tot[(corrs_tot['Day'] == day)].sort_values('total trials')        
                if len(plt_data['animal'].unique()) > 2:
                    xdata = plt_data['total trials']
                    ydata = plt_data['lick']
                    res = stats.linregress(xdata, y=ydata)
                    popt = (res.intercept, res.slope)
                    ax[i].plot(plt_data['total trials'], plt_data['lick'], 'ko')
                    ax[i].plot(plt_data['total trials'], lin(plt_data['total trials'], *popt), 'r-', lw=1)
                    print('%s %s -  slope: %5.3f, y-int: %5.3f,R:%1.5f, pval:%1.3E' % (c,day,res.slope, res.intercept,res.rvalue, res.pvalue))
                    ax[i].set_title(day)  
                    i+=1
            f.suptitle(c)
            f.tight_layout()
                    
    else:
        print(c)

corrs_tot = corrs[(corrs['stimulus'] == 'blank') & (corrs['Day'] == "SAT 1")].sort_values('total trials')
xdata = corrs_tot['total trials']
ydata = corrs_tot['lick']
res = stats.linregress(xdata, y=ydata)
popt = (res.intercept, res.slope)
f, axs = plt.subplots(1,1, figsize=(5,5))
axs.plot(corrs_tot['total trials'], corrs_tot['lick'], 'ko', label='data')
axs.plot(corrs_tot['total trials'],lin(corrs_tot['total trials'], *popt), 'r-',lw=1)
print(res)


ITI vs performance (no correlation)


In [ ]:
def lin(x, b, a):
    return a*x + b

def pw(x, a, b):
    return np.exp(a)*(x**b)



conds = corrs['cond'].unique()
for c in conds:
    #if 'lesion' in c:
        corrs_tot = corrs[(corrs['stimulus'] == 'stimulus') & (corrs['cond'] == c)]
        if len(corrs_tot['animal'].unique()) > 2:
            days = corrs_tot['Day'].unique()
            f, ax = plt.subplots(1,len(days), figsize=(5*(len(days)),5))
            i=0
            for day in days:
                plt_data = corrs_tot[(corrs_tot['Day'] == day)].sort_values('ITIlast20')        
                if len(plt_data['animal'].unique()) > 2:
                    xdata = plt_data['ITIlast20']
                    ydata = plt_data['perf']
                    res = stats.linregress(xdata, y=ydata)
                    popt = (res.intercept, res.slope)
                    ax[i].plot(plt_data['ITIlast20'], plt_data['perf'], 'ko')
                    ax[i].plot(plt_data['ITIlast20'], lin(plt_data['ITIlast20'], *popt), 'r-', lw=1)
                    print('%s %s -  slope: %5.3f, y-int: %5.3f,R:%1.5f, pval:%1.3E' % (c,day,res.slope, res.intercept,res.rvalue, res.pvalue))
                    ax[i].set_title(day)  
                    i+=1
            f.suptitle(c)
            f.tight_layout()
                    
        else:
            print(c)

corrs_tot = corrs[(corrs['stimulus'] == 'stimulus') & (corrs['Day'] == "SAT 1")].sort_values('ITIlast20')
xdata = corrs_tot['ITIlast20']
ydata = corrs_tot['perf']
res = stats.linregress(xdata, y=ydata)
popt = (res.intercept, res.slope)
f, axs = plt.subplots(1,1, figsize=(5,5))
axs.plot(corrs_tot['ITIlast20'], corrs_tot['perf'], 'ko', label='data')
axs.plot(corrs_tot['ITIlast20'],lin(corrs_tot['ITIlast20'], *popt), 'r-',lw=1)
print(res)

## ML approaches

In [ ]:
import sklearn
from sklearn import metrics, svm, neighbors, cluster
import sklearn.model_selection as ms

In [ ]:
keys

In [ ]:
#keys['millis bin'] = None
#keys['time bin'] = 'day_delta'
ant_data['Day'] = ant_data['day_delta'].to_numpy('timedelta64[D]').astype(float)
first20ant = get_nth_part_day(ant_data[~(ant_data['condition'] == 'retrain - retrain')], 5,1)
first20ant_lickfreq, first20ant_counts, first20ant_perf = agg(first20ant, keys, index, antkeep, values)

first20ant_perf = first20ant_perf.rename(columns={'lick':'perf'})
first20ant_counts = sum_trials(first20ant_counts, index + ["Day"], [],"trial no")

first20ant_lickfreq = first20ant_lickfreq.set_index(['condition', 'animal', 'Day', 'stimulus'])
first20ant_lickfreq = first20ant_lickfreq.unstack(level='stimulus')['lick'].rename(columns={'blank':'ant lick blank', 'stimulus': 'ant lick stimulus'})

first20ant_counts = first20ant_counts.set_index(['condition', 'animal', 'Day'])
first20ant_perf = first20ant_perf.set_index(['condition', 'animal', 'Day'])

first20iti = get_nth_part_day(diffs_by_time[(diffs_by_time['diff num'] == 1) & ~(diffs_by_time['condition'] == 'retrain - retrain')], 5,1)
first20miti = first20iti.set_index(['condition', 'animal', 'trial no'])
first20miti = first20miti.groupby(['condition','animal', 'Day'])['ITI'].apply(lambda x: pd.Series([x.mean(),x.std()], index=['iti mean','iti stdev'], name='measures'))
first20miti.index.rename({None:'measures'}, inplace=True)
first20miti = first20miti.unstack(level='measures')

first20ipi = get_nth_part_day(pokestarts[~(pokestarts['condition'] == 'retrain - retrain')], 5,1)
first20mipi = first20ipi.set_index(['condition', 'animal', 'trial no'])
first20mipi = first20mipi.groupby(['condition','animal', 'Day'])['ipi'].apply(lambda x: pd.Series([x.mean(),x.std()], index=['ipi mean','ipi stdev'], name='measures'))
first20mipi.index.rename({None:'measures'}, inplace=True)
first20mipi = first20mipi.unstack(level='measures')

In [ ]:
#keys['millis bin'] = None
#keys['time bin'] = 'day_delta'
ant_data = ant_data
ant_data['Day'] = ant_data['day_delta'].to_numpy('timedelta64[D]').astype(float)
last20ant = get_nth_part_day(ant_data[~(ant_data['condition'] == 'retrain - retrain')], 5,5)
last20ant_lickfreq, last20ant_counts, last20ant_perf = agg(last20ant, keys, index, antkeep, values)

last20ant_perf = last20ant_perf.rename(columns={'lick':'perf'})
last20ant_counts = sum_trials(last20ant_counts, index + ["Day"], [],"trial no")

last20ant_lickfreq = last20ant_lickfreq.set_index(['condition', 'animal', 'Day', 'stimulus'])
last20ant_lickfreq = last20ant_lickfreq.unstack(level='stimulus')['lick'].rename(columns={'blank':'ant lick blank', 'stimulus': 'ant lick stimulus'})

last20ant_counts = last20ant_counts.set_index(['condition', 'animal', 'Day'])
last20ant_perf = last20ant_perf.set_index(['condition', 'animal', 'Day'])

last20iti = get_nth_part_day(diffs_by_time[(diffs_by_time['diff num'] == 1) & ~(diffs_by_time['condition'] == 'retrain - retrain')], 5,5)
last20miti = last20iti.set_index(['condition', 'animal', 'trial no'])
last20miti = last20miti.groupby(['condition','animal', 'Day'])['ITI'].apply(lambda x: pd.Series([x.mean(),x.std()], index=['iti mean','iti stdev'], name='measures'))
last20miti.index.rename({None:'measures'}, inplace=True)
last20miti = last20miti.unstack(level='measures')

last20ipi = get_nth_part_day(pokestarts[~(pokestarts['condition'] == 'retrain - retrain')], 5,5)
last20mipi = last20ipi.set_index(['condition', 'animal', 'trial no'])
last20mipi = last20mipi.groupby(['condition','animal', 'Day'])['ipi'].apply(lambda x: pd.Series([x.mean(),x.std()], index=['ipi mean','ipi stdev'], name='measures'))
last20mipi.index.rename({None:'measures'}, inplace=True)
last20mipi = last20mipi.unstack(level='measures')

In [ ]:
first20 = pd.concat([first20ant_counts, first20ant_lickfreq,first20ant_perf['perf'],first20miti,first20mipi], axis=1)
first20 = first20.dropna()
first20 = first20.reset_index()
last20 = pd.concat([last20ant_counts, last20ant_lickfreq,last20ant_perf['perf'],last20miti,last20mipi], axis=1)
last20 = last20.reset_index()
last20 = last20.dropna()

In [ ]:
pred = last20[last20['Day'] == 0].set_index(['condition', 'animal'])['perf']
pred

In [ ]:
trf = first20[(first20['Day'] == -1) | (first20['Day'] == 0)]
trf['time'] = 'first20'
trl = last20[last20['Day']==-1]
trl['time'] = 'last20'
tr = pd.concat([trf, trl])
tr = tr.set_index(['condition', 'animal', 'Day', 'time']).unstack(level=['Day', 'time'])
tr = tr.loc[pred.index]
tr = tr.dropna()
pred = pred.loc[tr.index]

In [ ]:
X_train, X_test, y_train, y_test = ms.train_test_split(tr, pred, test_size=0.2, shuffle=False)

In [ ]:
clf = neighbors.KNeighborsRegressor()

In [ ]:
clf.fit(X_train, y_train)

In [ ]:
predicted = clf.predict(X_test)
print(metrics.max_error(y_test, predicted))
plt.plot(y_test, predicted, 'bo')
plt.plot(y_test, y_test, 'r-', lw=1)